In [3]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Configuration Variables (Modify as needed)
AOV = 1500  # Average Order Value
CHANNEL_COSTS = {
    'Email': 0.02,
    'Push': 0.01, 
    'SMS': 0.11,
    'WhatsApp': 0.11
}

def load_and_process_campaign_data(file_path):
    """
    Load campaign data and perform comprehensive analysis
    """
    # Load the data
    df = pd.read_csv(file_path)
    
    # Basic data cleaning
    df = df.fillna(0)
    
    # Convert numeric columns
    numeric_columns = ['Sent', 'Delivered', 'Unique Impressions', 'Unique Clicks', 
                      'Unique Conversions', 'Unique Click-Through Conversions']
    
    for col in numeric_columns:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', ''), errors='coerce').fillna(0)
    
    return df

def calculate_campaign_metrics(df):
    """
    Calculate all campaign performance metrics
    """
    # Group by Journey Name for analysis
    grouped = df.groupby('Journey Name').agg({
        'Campaign ID': 'count',
        'Sent': 'sum',
        'Delivered': 'sum', 
        'Unique Impressions': 'sum',
        'Unique Clicks': 'sum',
        'Unique Conversions': 'sum',
        'Unique Click-Through Conversions': 'sum'
    }).reset_index()
    
    # Rename columns for clarity
    grouped.rename(columns={'Campaign ID': 'Count_of_Campaign_ID'}, inplace=True)
    
    # Calculate performance metrics
    grouped['v_dr'] = np.where(grouped['Sent'] > 0, 
                              (grouped['Delivered'] / grouped['Sent'] * 100).round(0), 0)
    
    grouped['v_ctr'] = np.where(grouped['Delivered'] > 0,
                               (grouped['Unique Clicks'] / grouped['Delivered'] * 100).round(1), 0)
    
    grouped['v_cr'] = np.where(grouped['Unique Clicks'] > 0,
                              (grouped['Unique Click-Through Conversions'] / grouped['Unique Clicks'] * 100).round(1), 0)
    
    grouped['v_ord_per_sent'] = np.where(grouped['Sent'] > 0,
                                        (grouped['Unique Click-Through Conversions'] / grouped['Sent'] * 100).round(2), 0)
    
    # Format percentage columns
    grouped['v_dr_formatted'] = grouped['v_dr'].astype(int).astype(str) + '%'
    grouped['v_ctr_formatted'] = grouped['v_ctr'].astype(str) + '%'
    grouped['v_cr_formatted'] = grouped['v_cr'].astype(str) + '%'
    grouped['v_ord_per_sent_formatted'] = grouped['v_ord_per_sent'].astype(str) + '%'
    
    return grouped

def calculate_channel_analysis(df):
    """
    Calculate channel-wise performance analysis
    """
    # Create channel analysis
    channel_analysis = df.groupby(['Journey Name', 'Channel']).agg({
        'Sent': 'sum',
        'Delivered': 'sum',
        'Unique Click-Through Conversions': 'sum'
    }).reset_index()
    
    # Pivot to get channel columns
    sent_pivot = channel_analysis.pivot(index='Journey Name', columns='Channel', values='Sent').fillna(0)
    delivered_pivot = channel_analysis.pivot(index='Journey Name', columns='Channel', values='Delivered').fillna(0)
    conversions_pivot = channel_analysis.pivot(index='Journey Name', columns='Channel', values='Unique Click-Through Conversions').fillna(0)
    
    # Ensure all channels are present
    all_channels = ['Email', 'Push', 'SMS', 'WhatsApp']
    for channel in all_channels:
        if channel not in sent_pivot.columns:
            sent_pivot[channel] = 0
        if channel not in delivered_pivot.columns:
            delivered_pivot[channel] = 0
        if channel not in conversions_pivot.columns:
            conversions_pivot[channel] = 0
    
    # Calculate totals
    sent_pivot['Total_Sent'] = sent_pivot.sum(axis=1)
    delivered_pivot['Total_Delivered'] = delivered_pivot.sum(axis=1)
    conversions_pivot['Total_Conversions'] = conversions_pivot.sum(axis=1)
    
    return sent_pivot, delivered_pivot, conversions_pivot

def calculate_financial_metrics(df, aov=AOV, channel_costs=CHANNEL_COSTS):
    """
    Calculate cost, GTV, and ROI metrics
    """
    financial_df = df.groupby('Journey Name').agg({
        'Sent': 'sum',
        'Channel': lambda x: list(x),
        'Unique Click-Through Conversions': 'sum'
    }).reset_index()
    
    # Calculate costs based on channel
    financial_df['Cost'] = 0
    for idx, row in financial_df.iterrows():
        channel_sent = df[df['Journey Name'] == row['Journey Name']].groupby('Channel')['Sent'].sum()
        total_cost = 0
        for channel, sent_count in channel_sent.items():
            if channel in channel_costs:
                total_cost += sent_count * channel_costs[channel]
        financial_df.at[idx, 'Cost'] = total_cost
    
    # Calculate GTV and ROI
    financial_df['GTV'] = financial_df['Unique Click-Through Conversions'] * aov
    financial_df['ROI'] = np.where(financial_df['Cost'] > 0,
                                  (financial_df['GTV'] / financial_df['Cost']).round(2), 0)
    
    # Format currency
    financial_df['Cost_Formatted'] = '₹ ' + financial_df['Cost'].apply(lambda x: f"{x:,.2f}")
    financial_df['ROI_Formatted'] = '₹ ' + financial_df['ROI'].astype(str)
    
    return financial_df

def create_summary_table(df):
    """
    Create the main summary table combining all metrics
    """
    # Get basic metrics
    metrics_df = calculate_campaign_metrics(df)
    
    # Get financial metrics
    financial_df = calculate_financial_metrics(df)
    
    # Merge the dataframes
    summary_df = metrics_df.merge(financial_df[['Journey Name', 'Cost', 'GTV', 'ROI', 'Cost_Formatted', 'ROI_Formatted']], 
                                 on='Journey Name', how='left')
    
    # Format numbers with commas
    numeric_cols = ['Count_of_Campaign_ID', 'Sent', 'Delivered', 'Unique Impressions', 
                   'Unique Clicks', 'Unique Click-Through Conversions']
    
    for col in numeric_cols:
        if col in summary_df.columns:
            summary_df[f'{col}_Formatted'] = summary_df[col].apply(lambda x: f"{int(x):,}" if pd.notna(x) else "0")
    
    # Create the final display table
    display_df = pd.DataFrame({
        'Row Labels': summary_df['Journey Name'],
        'Count of Campaign ID': summary_df['Count_of_Campaign_ID_Formatted'],
        'Sum of Sent': summary_df['Sent_Formatted'],
        'Sum of Delivered': summary_df['Delivered_Formatted'],
        'Sum of Unique Impressions': summary_df['Unique Impressions_Formatted'],
        'Sum of Unique Clicks': summary_df['Unique Clicks_Formatted'],
        'Sum of Unique Click-Through Conversions': summary_df['Unique Click-Through Conversions_Formatted'],
        'v_dr': summary_df['v_dr_formatted'],
        'v_ctr': summary_df['v_ctr_formatted'],
        'v_cr': summary_df['v_cr_formatted'],
        'v_ord_per_sent': summary_df['v_ord_per_sent_formatted'],
        'Cost': summary_df['Cost_Formatted'],
        'GTV': summary_df['GTV'].apply(lambda x: f"{int(x):,}" if pd.notna(x) else "0"),
        'ROI': summary_df['ROI_Formatted']
    })
    
    return display_df, summary_df

def create_channel_breakdown_table(df):
    """
    Create channel-wise breakdown table
    """
    sent_pivot, delivered_pivot, conversions_pivot = calculate_channel_analysis(df)
    
    # Merge all pivots
    channel_df = sent_pivot.merge(delivered_pivot, left_index=True, right_index=True, suffixes=('_Sent', '_Delivered'))
    channel_df = channel_df.merge(conversions_pivot, left_index=True, right_index=True)
    
    # Rename columns for clarity
    channel_columns = {}
    for channel in ['Email', 'Push', 'SMS', 'WhatsApp']:
        if f'{channel}_Sent' in channel_df.columns:
            channel_columns[f'{channel}_Sent'] = f'{channel}_Sent'
        if f'{channel}_Delivered' in channel_df.columns:
            channel_columns[f'{channel}_Delivered'] = f'{channel}_Delivered'
        if channel in channel_df.columns:
            channel_columns[channel] = f'{channel}_Conversions'
    
    return channel_df

def analyze_campaign_report(file_path):
    """
    Main function to analyze campaign report and generate insights
    """
    print("📊 CAMPAIGN PERFORMANCE ANALYSIS")
    print("=" * 50)
    
    # Load data
    print("\n🔄 Loading and processing data...")
    df = load_and_process_campaign_data(file_path)
    print(f"✅ Loaded {len(df)} campaign records")
    
    # Generate summary table
    print("\n📋 Generating summary metrics...")
    display_df, summary_df = create_summary_table(df)
    
    # Display key insights
    print("\n🎯 KEY INSIGHTS:")
    print("-" * 30)
    
    total_sent = summary_df['Sent'].sum()
    total_delivered = summary_df['Delivered'].sum()
    total_conversions = summary_df['Unique Click-Through Conversions'].sum()
    total_cost = summary_df['Cost'].sum()
    total_gtv = summary_df['GTV'].sum()
    overall_roi = total_gtv / total_cost if total_cost > 0 else 0
    
    print(f"Total Messages Sent: {total_sent:,}")
    print(f"Total Messages Delivered: {total_delivered:,}")
    print(f"Total Conversions: {total_conversions:,}")
    print(f"Total Cost: ₹{total_cost:,.2f}")
    print(f"Total GTV: ₹{total_gtv:,.2f}")
    print(f"Overall ROI: {overall_roi:.2f}x")
    print(f"Overall Delivery Rate: {(total_delivered/total_sent*100):.1f}%")
    print(f"Overall Conversion Rate: {(total_conversions/total_delivered*100):.2f}%")
    
    # Top performers
    print("\n🏆 TOP PERFORMING JOURNEYS:")
    print("-" * 35)
    top_roi = summary_df.nlargest(5, 'ROI')[['Journey Name', 'ROI', 'GTV']]
    for _, row in top_roi.iterrows():
        print(f"{row['Journey Name']}: ROI {row['ROI']:.2f}x, GTV ₹{row['GTV']:,.0f}")
    
    return display_df, summary_df, df

# Usage Example
def main():
    """
    Main execution function
    """
    # File path - modify this to your file location
    file_path = "report-1750749443470_jBRS6FP_Snapmint _in~~c2ab3517.csv"
    
    try:
        # Run analysis
        display_df, summary_df, raw_df = analyze_campaign_report(file_path)
        
        # Display the main summary table
        print("\n📊 CAMPAIGN SUMMARY TABLE:")
        print("=" * 100)
        print(display_df.to_string(index=False))
        
        # Additional analysis options
        print("\n\n🔍 ADDITIONAL ANALYSIS OPTIONS:")
        print("1. Channel-wise breakdown")
        print("2. Journey performance comparison")
        print("3. Cost efficiency analysis")
        
        return display_df, summary_df, raw_df
        
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        print("Please check your file path and data format.")
        return None, None, None

# Run the analysis
if __name__ == "__main__":
    display_table, summary_data, raw_data = main()

📊 CAMPAIGN PERFORMANCE ANALYSIS

🔄 Loading and processing data...
✅ Loaded 28750 campaign records

📋 Generating summary metrics...

🎯 KEY INSIGHTS:
------------------------------
Total Messages Sent: 80,240,147
Total Messages Delivered: 74,464,766
Total Conversions: 82,348
Total Cost: ₹4,228,022.72
Total GTV: ₹123,522,000.00
Overall ROI: 29.22x
Overall Delivery Rate: 92.8%
Overall Conversion Rate: 0.11%

🏆 TOP PERFORMING JOURNEYS:
-----------------------------------
Ixigo app download: ROI 295.31x, GTV ₹1,743,000
Product Viewed No Plan Selected (Pan Not Given): ROI 283.83x, GTV ₹32,776,500
OA CC but no DP: ROI 135.22x, GTV ₹1,063,500
NU ALL CC but no DP: ROI 101.24x, GTV ₹7,101,000
Online Mobile_PL Eligible_27th May: ROI 93.23x, GTV ₹772,500

📊 CAMPAIGN SUMMARY TABLE:
                                     Row Labels Count of Campaign ID Sum of Sent Sum of Delivered Sum of Unique Impressions Sum of Unique Clicks Sum of Unique Click-Through Conversions v_dr v_ctr  v_cr v_ord_per_sent     

In [ ]:
import pandas as pd
import numpy as np
import warnings
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import dash
from dash import dcc, html, dash_table, callback
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc

warnings.filterwarnings('ignore')

# Configuration Variables (Modify as needed)
AOV = 1500  # Average Order Value
CHANNEL_COSTS = {
    'Email': 0.02,
    'Push': 0.01, 
    'SMS': 0.11,
    'WhatsApp': 0.11
}

class CampaignAnalyzer:
    def __init__(self, file_path, aov=AOV, channel_costs=CHANNEL_COSTS):
        self.file_path = file_path
        self.aov = aov
        self.channel_costs = channel_costs
        self.raw_data = None
        self.summary_data = None
        self.channel_data = None
        
    def load_and_process_data(self):
        """Load and process campaign data"""
        # Load the data
        df = pd.read_csv(self.file_path)
        
        # Basic data cleaning
        df = df.fillna(0)
        
        # Convert numeric columns
        numeric_columns = ['Sent', 'Delivered', 'Unique Impressions', 'Unique Clicks', 
                          'Unique Conversions', 'Unique Click-Through Conversions']
        
        for col in numeric_columns:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', ''), errors='coerce').fillna(0)
        
        self.raw_data = df
        return df
    
    def calculate_summary_metrics(self):
        """Calculate main summary table metrics"""
        df = self.raw_data
        
        # Group by Journey Name for analysis
        grouped = df.groupby('Journey Name').agg({
            'Campaign ID': 'count',
            'Sent': 'sum',
            'Delivered': 'sum', 
            'Unique Impressions': 'sum',
            'Unique Clicks': 'sum',
            'Unique Conversions': 'sum',
            'Unique Click-Through Conversions': 'sum'
        }).reset_index()
        
        # Rename columns for clarity
        grouped.rename(columns={'Campaign ID': 'Count of Campaign ID'}, inplace=True)
        
        # Calculate performance metrics
        grouped['v_dr'] = np.where(grouped['Sent'] > 0, 
                                  (grouped['Delivered'] / grouped['Sent'] * 100).round(0), 0)
        
        grouped['v_ctr'] = np.where(grouped['Delivered'] > 0,
                                   (grouped['Unique Clicks'] / grouped['Delivered'] * 100).round(1), 0)
        
        grouped['v_cr'] = np.where(grouped['Unique Clicks'] > 0,
                                  (grouped['Unique Click-Through Conversions'] / grouped['Unique Clicks'] * 100).round(1), 0)
        
        grouped['v_ord_per_sent'] = np.where(grouped['Sent'] > 0,
                                            (grouped['Unique Click-Through Conversions'] / grouped['Sent'] * 100).round(2), 0)
        
        # Calculate financial metrics
        financial_metrics = self.calculate_financial_metrics()
        
        # Merge financial data
        summary_df = grouped.merge(financial_metrics[['Journey Name', 'Cost', 'GTV', 'ROI']], 
                                 on='Journey Name', how='left')
        
        # Format the data for display
        summary_df['Row Labels'] = summary_df['Journey Name']
        summary_df['Sum of Sent'] = summary_df['Sent'].apply(lambda x: f"{int(x):,}")
        summary_df['Sum of Delivered'] = summary_df['Delivered'].apply(lambda x: f"{int(x):,}")
        summary_df['Sum of Unique Impressions'] = summary_df['Unique Impressions'].apply(lambda x: f"{int(x):,}")
        summary_df['Sum of Unique Clicks'] = summary_df['Unique Clicks'].apply(lambda x: f"{int(x):,}")
        summary_df['Sum of Unique Click-Through Conversions'] = summary_df['Unique Click-Through Conversions'].apply(lambda x: f"{int(x):,}")
        summary_df['v_dr_formatted'] = summary_df['v_dr'].astype(int).astype(str) + '%'
        summary_df['v_ctr_formatted'] = summary_df['v_ctr'].astype(str) + '%'
        summary_df['v_cr_formatted'] = summary_df['v_cr'].astype(str) + '%'
        summary_df['v_ord_per_sent_formatted'] = summary_df['v_ord_per_sent'].astype(str) + '%'
        summary_df['Cost_formatted'] = summary_df['Cost'].apply(lambda x: f"₹ {x:,.2f}")
        summary_df['GTV_formatted'] = summary_df['GTV'].apply(lambda x: f"{int(x):,}")
        summary_df['ROI_formatted'] = summary_df['ROI'].apply(lambda x: f"₹ {x:.2f}")
        
        self.summary_data = summary_df
        return summary_df
    
    def calculate_financial_metrics(self):
        """Calculate cost, GTV, and ROI metrics"""
        df = self.raw_data
        
        financial_df = df.groupby('Journey Name').agg({
            'Sent': 'sum',
            'Channel': lambda x: list(x),
            'Unique Click-Through Conversions': 'sum'
        }).reset_index()
        
        # Calculate costs based on channel
        financial_df['Cost'] = 0
        for idx, row in financial_df.iterrows():
            channel_sent = df[df['Journey Name'] == row['Journey Name']].groupby('Channel')['Sent'].sum()
            total_cost = 0
            for channel, sent_count in channel_sent.items():
                if channel in self.channel_costs:
                    total_cost += sent_count * self.channel_costs[channel]
            financial_df.at[idx, 'Cost'] = total_cost
        
        # Calculate GTV and ROI
        financial_df['GTV'] = financial_df['Unique Click-Through Conversions'] * self.aov
        financial_df['ROI'] = np.where(financial_df['Cost'] > 0,
                                      (financial_df['GTV'] / financial_df['Cost']).round(2), 0)
        
        return financial_df
    
    def calculate_channel_breakdown(self):
        """Calculate channel-wise breakdown"""
        df = self.raw_data
        
        # Create channel analysis
        channel_analysis = df.groupby(['Journey Name', 'Channel']).agg({
            'Sent': 'sum',
            'Delivered': 'sum',
            'Unique Click-Through Conversions': 'sum'
        }).reset_index()
        
        # Pivot tables for different metrics
        sent_pivot = channel_analysis.pivot(index='Journey Name', columns='Channel', values='Sent').fillna(0)
        delivered_pivot = channel_analysis.pivot(index='Journey Name', columns='Channel', values='Delivered').fillna(0)
        conversions_pivot = channel_analysis.pivot(index='Journey Name', columns='Channel', values='Unique Click-Through Conversions').fillna(0)
        
        # Ensure all channels are present
        all_channels = ['Email', 'Push', 'SMS', 'WhatsApp']
        for channel in all_channels:
            if channel not in sent_pivot.columns:
                sent_pivot[channel] = 0
            if channel not in delivered_pivot.columns:
                delivered_pivot[channel] = 0
            if channel not in conversions_pivot.columns:
                conversions_pivot[channel] = 0
        
        # Calculate totals
        sent_pivot['Total'] = sent_pivot[all_channels].sum(axis=1)
        delivered_pivot['Total'] = delivered_pivot[all_channels].sum(axis=1)
        conversions_pivot['Total'] = conversions_pivot[all_channels].sum(axis=1)
        
        # Create comprehensive channel breakdown
        channel_breakdown = pd.DataFrame()
        channel_breakdown['Row Labels'] = sent_pivot.index
        
        # Add sent data
        for channel in all_channels:
            channel_breakdown[f'{channel}_Sent'] = sent_pivot[channel].apply(lambda x: f"{int(x):,}" if x > 0 else "")
        
        # Add delivered data  
        for channel in all_channels:
            channel_breakdown[f'{channel}_Delivered'] = delivered_pivot[channel].apply(lambda x: f"{int(x):,}" if x > 0 else "")
        
        # Add conversions data
        for channel in all_channels:
            channel_breakdown[f'{channel}_Conversions'] = conversions_pivot[channel].apply(lambda x: f"{int(x):,}" if x > 0 else "")
        
        # Add totals
        channel_breakdown['Total_Sent'] = sent_pivot['Total'].apply(lambda x: f"{int(x):,}")
        channel_breakdown['Total_Delivered'] = delivered_pivot['Total'].apply(lambda x: f"{int(x):,}")
        channel_breakdown['Total_Conversions'] = conversions_pivot['Total'].apply(lambda x: f"{int(x):,}")
        
        self.channel_data = channel_breakdown
        return channel_breakdown

# Initialize the analyzer
analyzer = CampaignAnalyzer("report-1750749443470_jBRS6FP_Snapmint _in~~c2ab3517.csv")

# Load and process data
analyzer.load_and_process_data()
summary_df = analyzer.calculate_summary_metrics()
channel_df = analyzer.calculate_channel_breakdown()

# Initialize Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define the layout
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.H1("Campaign Performance Analytics Dashboard", 
                   className="text-center mb-4 text-primary"),
            html.Hr()
        ])
    ]),
    
    # Key Metrics Row
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H4(f"{summary_df['Sent'].sum():,}", className="text-primary"),
                    html.P("Total Messages Sent", className="mb-0")
                ])
            ])
        ], width=2),
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H4(f"{summary_df['Delivered'].sum():,}", className="text-success"),
                    html.P("Total Delivered", className="mb-0")
                ])
            ])
        ], width=2),
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H4(f"{summary_df['Unique Click-Through Conversions'].sum():,}", className="text-info"),
                    html.P("Total Conversions", className="mb-0")
                ])
            ])
        ], width=2),
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H4(f"₹{summary_df['Cost'].sum():,.2f}", className="text-warning"),
                    html.P("Total Cost", className="mb-0")
                ])
            ])
        ], width=2),
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H4(f"₹{summary_df['GTV'].sum():,.0f}", className="text-success"),
                    html.P("Total GTV", className="mb-0")
                ])
            ])
        ], width=2),
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H4(f"{(summary_df['GTV'].sum()/summary_df['Cost'].sum()):.2f}x", className="text-danger"),
                    html.P("Overall ROI", className="mb-0")
                ])
            ])
        ], width=2),
    ], className="mb-4"),
    
    # Tabs for different views
    dbc.Tabs([
        dbc.Tab(label="Summary Table", tab_id="summary"),
        dbc.Tab(label="Channel Breakdown", tab_id="channels"),
        dbc.Tab(label="Performance Charts", tab_id="charts"),
    ], id="tabs", active_tab="summary"),
    
    html.Div(id="tab-content", className="mt-4")
], fluid=True)

@app.callback(
    Output("tab-content", "children"),
    Input("tabs", "active_tab")
)
def render_tab_content(active_tab):
    if active_tab == "summary":
        # Create summary table
        summary_display_df = summary_df[[
            'Row Labels', 'Count of Campaign ID', 'Sum of Sent', 'Sum of Delivered',
            'Sum of Unique Impressions', 'Sum of Unique Clicks', 
            'Sum of Unique Click-Through Conversions', 'v_dr_formatted', 
            'v_ctr_formatted', 'v_cr_formatted', 'v_ord_per_sent_formatted',
            'Cost_formatted', 'GTV_formatted', 'ROI_formatted'
        ]].copy()
        
        summary_display_df.columns = [
            'Row Labels', 'Count of Campaign ID', 'Sum of Sent', 'Sum of Delivered',
            'Sum of Unique Impressions', 'Sum of Unique Clicks', 
            'Sum of Unique Click-Through Conversions', 'v_dr', 'v_ctr', 'v_cr',
            'v_ord_per_sent', 'Cost', 'GTV', 'ROI'
        ]
        
        return dash_table.DataTable(
            data=summary_display_df.to_dict('records'),
            columns=[{"name": i, "id": i} for i in summary_display_df.columns],
            style_table={'overflowX': 'auto'},
            style_cell={
                'textAlign': 'left',
                'padding': '10px',
                'fontFamily': 'Arial'
            },
            style_header={
                'backgroundColor': 'rgb(230, 230, 230)',
                'fontWeight': 'bold'
            },
            style_data_conditional=[
                {
                    'if': {'row_index': 'odd'},
                    'backgroundColor': 'rgb(248, 248, 248)'
                }
            ],
            sort_action="native",
            filter_action="native",
            page_size=20,
            export_format="xlsx",
            export_headers="display"
        )
    
    elif active_tab == "channels":
        return dash_table.DataTable(
            data=channel_df.to_dict('records'),
            columns=[{"name": i, "id": i} for i in channel_df.columns],
            style_table={'overflowX': 'auto'},
            style_cell={
                'textAlign': 'left',
                'padding': '10px',
                'fontFamily': 'Arial',
                'minWidth': '100px'
            },
            style_header={
                'backgroundColor': 'rgb(230, 230, 230)',
                'fontWeight': 'bold'
            },
            style_data_conditional=[
                {
                    'if': {'row_index': 'odd'},
                    'backgroundColor': 'rgb(248, 248, 248)'
                }
            ],
            sort_action="native",
            filter_action="native",
            page_size=15,
            export_format="xlsx",
            export_headers="display"
        )
    
    elif active_tab == "charts":
        # Create performance charts
        fig1 = px.bar(summary_df.nlargest(10, 'ROI'), 
                     x='Journey Name', y='ROI', 
                     title="Top 10 Journeys by ROI")
        fig1.update_xaxis(tickangle=45)
        
        fig2 = px.scatter(summary_df, 
                         x='Cost', y='GTV', 
                         size='Unique Click-Through Conversions',
                         hover_name='Journey Name',
                         title="Cost vs GTV Analysis")
        
        fig3 = px.pie(summary_df.nlargest(8, 'Sent'), 
                     values='Sent', names='Journey Name',
                     title="Top 8 Journeys by Messages Sent")
        
        return dbc.Row([
            dbc.Col([dcc.Graph(figure=fig1)], width=6),
            dbc.Col([dcc.Graph(figure=fig2)], width=6),
            dbc.Col([dcc.Graph(figure=fig3)], width=12)
        ])

if __name__ == "__main__":
    print("🚀 Starting Campaign Analytics Dashboard...")
    print("📊 Dashboard will be available at: http://127.0.0.1:8050")
    print("📋 Features available:")
    print("   - Interactive Summary Table (sortable, filterable, exportable)")
    print("   - Channel Breakdown Analysis")
    print("   - Performance Visualization Charts")
    print("   - Real-time metrics calculations")
    
    app.run(debug=True, port=8050)

🚀 Starting Campaign Analytics Dashboard...
📊 Dashboard will be available at: http://127.0.0.1:8050
📋 Features available:
   - Interactive Summary Table (sortable, filterable, exportable)
   - Channel Breakdown Analysis
   - Performance Visualization Charts
   - Real-time metrics calculations


---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
Cell In[6], line 353, in render_tab_content(active_tab='charts')
    348 elif active_tab == "charts":
    349     # Create performance charts
    350     fig1 = px.bar(summary_df.nlargest(10, 'ROI'), 
    351                  x='Journey Name', y='ROI', 
    352                  title="Top 10 Journeys by ROI")
--> 353     fig1.update_xaxis(tickangle=45)
        fig1 = Figure({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': 'Journey Name=%{x}<br>ROI=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': '',
              'offsetgroup': '',
              'orientation': 'v',
              'showlegend': False,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['Ixigo app download', 'P

In [ ]:
import pandas as pd
import numpy as np
import warnings
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import dash
from dash import dcc, html, dash_table, callback
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc

warnings.filterwarnings('ignore')

# Configuration Variables (Modify as needed)
AOV = 1500  # Average Order Value
CHANNEL_COSTS = {
    'Email': 0.02,
    'Push': 0.01, 
    'SMS': 0.11,
    'WhatsApp': 0.11
}

class CampaignAnalyzer:
    def __init__(self, file_path, aov=AOV, channel_costs=CHANNEL_COSTS):
        self.file_path = file_path
        self.aov = aov
        self.channel_costs = channel_costs
        self.raw_data = None
        self.summary_data = None
        self.channel_data = None
        
    def load_and_process_data(self):
        """Load and process campaign data"""
        try:
            # Load the data
            df = pd.read_csv(self.file_path)
            
            # Basic data cleaning
            df = df.fillna(0)
            
            # Convert numeric columns
            numeric_columns = ['Sent', 'Delivered', 'Unique Impressions', 'Unique Clicks', 
                              'Unique Conversions', 'Unique Click-Through Conversions']
            
            for col in numeric_columns:
                if col in df.columns:
                    df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', ''), errors='coerce').fillna(0)
            
            self.raw_data = df
            return df
        except Exception as e:
            print(f"Error loading data: {e}")
            return None
    
    def calculate_summary_metrics(self):
        """Calculate main summary table metrics"""
        if self.raw_data is None:
            return None
            
        df = self.raw_data
        
        # Group by Journey Name for analysis
        grouped = df.groupby('Journey Name').agg({
            'Campaign ID': 'count',
            'Sent': 'sum',
            'Delivered': 'sum', 
            'Unique Impressions': 'sum',
            'Unique Clicks': 'sum',
            'Unique Conversions': 'sum',
            'Unique Click-Through Conversions': 'sum'
        }).reset_index()
        
        # Rename columns for clarity
        grouped.rename(columns={'Campaign ID': 'Count of Campaign ID'}, inplace=True)
        
        # Calculate performance metrics
        grouped['v_dr'] = np.where(grouped['Sent'] > 0, 
                                  (grouped['Delivered'] / grouped['Sent'] * 100).round(0), 0)
        
        grouped['v_ctr'] = np.where(grouped['Delivered'] > 0,
                                   (grouped['Unique Clicks'] / grouped['Delivered'] * 100).round(1), 0)
        
        grouped['v_cr'] = np.where(grouped['Unique Clicks'] > 0,
                                  (grouped['Unique Click-Through Conversions'] / grouped['Unique Clicks'] * 100).round(1), 0)
        
        grouped['v_ord_per_sent'] = np.where(grouped['Sent'] > 0,
                                            (grouped['Unique Click-Through Conversions'] / grouped['Sent'] * 100).round(2), 0)
        
        # Calculate financial metrics
        financial_metrics = self.calculate_financial_metrics()
        
        # Merge financial data
        summary_df = grouped.merge(financial_metrics[['Journey Name', 'Cost', 'GTV', 'ROI']], 
                                 on='Journey Name', how='left')
        
        # Format the data for display
        summary_df['Row Labels'] = summary_df['Journey Name']
        summary_df['Sum of Sent'] = summary_df['Sent'].apply(lambda x: f"{int(x):,}")
        summary_df['Sum of Delivered'] = summary_df['Delivered'].apply(lambda x: f"{int(x):,}")
        summary_df['Sum of Unique Impressions'] = summary_df['Unique Impressions'].apply(lambda x: f"{int(x):,}")
        summary_df['Sum of Unique Clicks'] = summary_df['Unique Clicks'].apply(lambda x: f"{int(x):,}")
        summary_df['Sum of Unique Click-Through Conversions'] = summary_df['Unique Click-Through Conversions'].apply(lambda x: f"{int(x):,}")
        summary_df['v_dr_formatted'] = summary_df['v_dr'].astype(int).astype(str) + '%'
        summary_df['v_ctr_formatted'] = summary_df['v_ctr'].astype(str) + '%'
        summary_df['v_cr_formatted'] = summary_df['v_cr'].astype(str) + '%'
        summary_df['v_ord_per_sent_formatted'] = summary_df['v_ord_per_sent'].astype(str) + '%'
        summary_df['Cost_formatted'] = summary_df['Cost'].apply(lambda x: f"₹ {x:,.2f}")
        summary_df['GTV_formatted'] = summary_df['GTV'].apply(lambda x: f"{int(x):,}")
        summary_df['ROI_formatted'] = summary_df['ROI'].apply(lambda x: f"₹ {x:.2f}")
        
        self.summary_data = summary_df
        return summary_df
    
    def calculate_financial_metrics(self):
        """Calculate cost, GTV, and ROI metrics"""
        if self.raw_data is None:
            return pd.DataFrame()
            
        df = self.raw_data
        
        financial_df = df.groupby('Journey Name').agg({
            'Sent': 'sum',
            'Channel': lambda x: list(x),
            'Unique Click-Through Conversions': 'sum'
        }).reset_index()
        
        # Calculate costs based on channel
        financial_df['Cost'] = 0
        for idx, row in financial_df.iterrows():
            channel_sent = df[df['Journey Name'] == row['Journey Name']].groupby('Channel')['Sent'].sum()
            total_cost = 0
            for channel, sent_count in channel_sent.items():
                if channel in self.channel_costs:
                    total_cost += sent_count * self.channel_costs[channel]
            financial_df.at[idx, 'Cost'] = total_cost
        
        # Calculate GTV and ROI
        financial_df['GTV'] = financial_df['Unique Click-Through Conversions'] * self.aov
        financial_df['ROI'] = np.where(financial_df['Cost'] > 0,
                                      (financial_df['GTV'] / financial_df['Cost']).round(2), 0)
        
        return financial_df
    
    def calculate_channel_breakdown(self):
        """Calculate channel-wise breakdown"""
        if self.raw_data is None:
            return pd.DataFrame()
            
        df = self.raw_data
        
        # Create channel analysis
        channel_analysis = df.groupby(['Journey Name', 'Channel']).agg({
            'Sent': 'sum',
            'Delivered': 'sum',
            'Unique Click-Through Conversions': 'sum'
        }).reset_index()
        
        # Create a comprehensive breakdown table similar to your reference
        journeys = df['Journey Name'].unique()
        channels = ['Email', 'Push', 'SMS', 'WhatsApp']
        
        # Initialize the breakdown dataframe
        breakdown_data = []
        
        for journey in journeys:
            journey_data = df[df['Journey Name'] == journey]
            row_data = {'Row Labels': journey}
            
            # Calculate totals
            total_sent = journey_data['Sent'].sum()
            total_delivered = journey_data['Delivered'].sum()
            total_conversions = journey_data['Unique Click-Through Conversions'].sum()
            
            # Add channel-wise data
            for channel in channels:
                channel_data = journey_data[journey_data['Channel'] == channel]
                sent = channel_data['Sent'].sum()
                delivered = channel_data['Delivered'].sum()
                conversions = channel_data['Unique Click-Through Conversions'].sum()
                
                row_data[f'{channel}_Sent'] = f"{int(sent):,}" if sent > 0 else ""
                row_data[f'{channel}_Delivered'] = f"{int(delivered):,}" if delivered > 0 else ""
                row_data[f'{channel}_Conversions'] = f"{int(conversions):,}" if conversions > 0 else ""
            
            # Add totals
            row_data['Total_Sent'] = f"{int(total_sent):,}"
            row_data['Total_Delivered'] = f"{int(total_delivered):,}"
            row_data['Total_Conversions'] = f"{int(total_conversions):,}"
            
            breakdown_data.append(row_data)
        
        channel_breakdown = pd.DataFrame(breakdown_data)
        self.channel_data = channel_breakdown
        return channel_breakdown

# Initialize the analyzer
try:
    analyzer = CampaignAnalyzer("report-1750749443470_jBRS6FP_Snapmint _in~~c2ab3517.csv")
    
    # Load and process data
    data_loaded = analyzer.load_and_process_data()
    
    if data_loaded is not None:
        summary_df = analyzer.calculate_summary_metrics()
        channel_df = analyzer.calculate_channel_breakdown()
        
        print(f"✅ Data loaded successfully: {len(data_loaded)} records")
        print(f"📊 Summary table: {len(summary_df)} journeys")
        print(f"📋 Channel breakdown: {len(channel_df)} journeys")
    else:
        print("❌ Failed to load data")
        summary_df = pd.DataFrame()
        channel_df = pd.DataFrame()
        
except Exception as e:
    print(f"❌ Error initializing analyzer: {e}")
    summary_df = pd.DataFrame()
    channel_df = pd.DataFrame()

# Initialize Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define the layout with error handling
def create_layout():
    if summary_df.empty:
        return dbc.Container([
            html.H1("Campaign Performance Analytics Dashboard", className="text-center mb-4 text-danger"),
            html.Hr(),
            dbc.Alert("Failed to load data. Please check your file path and data format.", color="danger")
        ], fluid=True)
    
    return dbc.Container([
        dbc.Row([
            dbc.Col([
                html.H1("Campaign Performance Analytics Dashboard", 
                       className="text-center mb-4 text-primary"),
                html.Hr()
            ])
        ]),
        
        # Key Metrics Row
        dbc.Row([
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H4(f"{summary_df['Sent'].sum():,}", className="text-primary"),
                        html.P("Total Messages Sent", className="mb-0")
                    ])
                ])
            ], width=2),
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H4(f"{summary_df['Delivered'].sum():,}", className="text-success"),
                        html.P("Total Delivered", className="mb-0")
                    ])
                ])
            ], width=2),
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H4(f"{summary_df['Unique Click-Through Conversions'].sum():,}", className="text-info"),
                        html.P("Total Conversions", className="mb-0")
                    ])
                ])
            ], width=2),
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H4(f"₹{summary_df['Cost'].sum():,.2f}", className="text-warning"),
                        html.P("Total Cost", className="mb-0")
                    ])
                ])
            ], width=2),
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H4(f"₹{summary_df['GTV'].sum():,.0f}", className="text-success"),
                        html.P("Total GTV", className="mb-0")
                    ])
                ])
            ], width=2),
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H4(f"{(summary_df['GTV'].sum()/summary_df['Cost'].sum() if summary_df['Cost'].sum() > 0 else 0):.2f}x", className="text-danger"),
                        html.P("Overall ROI", className="mb-0")
                    ])
                ])
            ], width=2),
        ], className="mb-4"),
        
        # Tabs for different views
        dbc.Tabs([
            dbc.Tab(label="Summary Table", tab_id="summary"),
            dbc.Tab(label="Channel Breakdown", tab_id="channels"),
            dbc.Tab(label="Performance Charts", tab_id="charts"),
        ], id="tabs", active_tab="summary"),
        
        html.Div(id="tab-content", className="mt-4")
    ], fluid=True)

app.layout = create_layout()

@app.callback(
    Output("tab-content", "children"),
    Input("tabs", "active_tab")
)
def render_tab_content(active_tab):
    if summary_df.empty:
        return dbc.Alert("No data available to display.", color="warning")
    
    try:
        if active_tab == "summary":
            # Create summary table
            summary_display_df = summary_df[[
                'Row Labels', 'Count of Campaign ID', 'Sum of Sent', 'Sum of Delivered',
                'Sum of Unique Impressions', 'Sum of Unique Clicks', 
                'Sum of Unique Click-Through Conversions', 'v_dr_formatted', 
                'v_ctr_formatted', 'v_cr_formatted', 'v_ord_per_sent_formatted',
                'Cost_formatted', 'GTV_formatted', 'ROI_formatted'
            ]].copy()
            
            summary_display_df.columns = [
                'Row Labels', 'Count of Campaign ID', 'Sum of Sent', 'Sum of Delivered',
                'Sum of Unique Impressions', 'Sum of Unique Clicks', 
                'Sum of Unique Click-Through Conversions', 'v_dr', 'v_ctr', 'v_cr',
                'v_ord_per_sent', 'Cost', 'GTV', 'ROI'
            ]
            
            return dash_table.DataTable(
                data=summary_display_df.to_dict('records'),
                columns=[{"name": i, "id": i} for i in summary_display_df.columns],
                style_table={'overflowX': 'auto'},
                style_cell={
                    'textAlign': 'left',
                    'padding': '10px',
                    'fontFamily': 'Arial',
                    'fontSize': '12px'
                },
                style_header={
                    'backgroundColor': 'rgb(230, 230, 230)',
                    'fontWeight': 'bold',
                    'fontSize': '14px'
                },
                style_data_conditional=[
                    {
                        'if': {'row_index': 'odd'},
                        'backgroundColor': 'rgb(248, 248, 248)'
                    }
                ],
                sort_action="native",
                filter_action="native",
                page_size=20,
                export_format="xlsx",
                export_headers="display"
            )
        
        elif active_tab == "channels":
            if channel_df.empty:
                return dbc.Alert("No channel breakdown data available.", color="warning")
                
            return dash_table.DataTable(
                data=channel_df.to_dict('records'),
                columns=[{"name": i, "id": i} for i in channel_df.columns],
                style_table={'overflowX': 'auto'},
                style_cell={
                    'textAlign': 'left',
                    'padding': '8px',
                    'fontFamily': 'Arial',
                    'fontSize': '11px',
                    'minWidth': '80px'
                },
                style_header={
                    'backgroundColor': 'rgb(230, 230, 230)',
                    'fontWeight': 'bold',
                    'fontSize': '12px'
                },
                style_data_conditional=[
                    {
                        'if': {'row_index': 'odd'},
                        'backgroundColor': 'rgb(248, 248, 248)'
                    }
                ],
                sort_action="native",
                filter_action="native",
                page_size=15,
                export_format="xlsx",
                export_headers="display"
            )
        
        elif active_tab == "charts":
            # Create performance charts with error handling
            try:
                # Filter data for meaningful charts
                chart_data = summary_df[summary_df['ROI'] > 0].nlargest(10, 'ROI')
                
                if chart_data.empty:
                    return dbc.Alert("No data available for charts.", color="warning")
                
                # Chart 1: Top ROI Performance
                fig1 = px.bar(chart_data, 
                             x='Journey Name', y='ROI', 
                             title="Top 10 Journeys by ROI",
                             labels={'ROI': 'Return on Investment'})
                fig1.update_layout(xaxis_tickangle=45, height=400)
                
                # Chart 2: Cost vs GTV Analysis
                scatter_data = summary_df[(summary_df['Cost'] > 0) & (summary_df['GTV'] > 0)]
                fig2 = px.scatter(scatter_data, 
                                 x='Cost', y='GTV', 
                                 size='Unique Click-Through Conversions',
                                 hover_name='Journey Name',
                                 title="Cost vs GTV Analysis",
                                 labels={'Cost': 'Total Cost (₹)', 'GTV': 'Gross Transaction Value (₹)'})
                fig2.update_layout(height=400)
                
                # Chart 3: Volume Distribution
                volume_data = summary_df.nlargest(8, 'Sent')
                fig3 = px.pie(volume_data, 
                             values='Sent', names='Journey Name',
                             title="Top 8 Journeys by Messages Sent")
                fig3.update_layout(height=400)
                
                # Chart 4: Channel Performance
                if not channel_df.empty:
                    # Calculate channel totals
                    channel_totals = {}
                    for channel in ['Email', 'Push', 'SMS', 'WhatsApp']:
                        total = 0
                        for _, row in channel_df.iterrows():
                            val = row.get(f'{channel}_Sent', '0')
                            if val and val != '':
                                total += int(val.replace(',', ''))
                        channel_totals[channel] = total
                    
                    fig4 = px.bar(x=list(channel_totals.keys()), 
                                 y=list(channel_totals.values()),
                                 title="Messages Sent by Channel",
                                 labels={'x': 'Channel', 'y': 'Total Messages Sent'})
                    fig4.update_layout(height=400)
                else:
                    fig4 = px.bar(title="Channel data not available")
                
                return dbc.Row([
                    dbc.Col([dcc.Graph(figure=fig1)], width=6),
                    dbc.Col([dcc.Graph(figure=fig2)], width=6),
                    dbc.Col([dcc.Graph(figure=fig3)], width=6),
                    dbc.Col([dcc.Graph(figure=fig4)], width=6)
                ])
                
            except Exception as e:
                return dbc.Alert(f"Error creating charts: {str(e)}", color="danger")
    
    except Exception as e:
        return dbc.Alert(f"Error rendering content: {str(e)}", color="danger")

if __name__ == "__main__":
    print("Starting Campaign Analytics Dashboard...")
    print("Dashboard will be available at: http://127.0.0.1:8051")
    print("Features available:")
    print("   - Interactive Summary Table (sortable, filterable, exportable)")
    print("   - Channel Breakdown Analysis")
    print("   - Performance Visualization Charts")
    print("   - Real-time metrics calculations")
    
    try:
        app.run(debug=True, port=8051)
    except Exception as e:
        print(f"Error starting server: {e}")

✅ Data loaded successfully: 28750 records
📊 Summary table: 21 journeys
📋 Channel breakdown: 21 journeys
🚀 Starting Campaign Analytics Dashboard...
📊 Dashboard will be available at: http://127.0.0.1:8050
📋 Features available:
   - Interactive Summary Table (sortable, filterable, exportable)
   - Channel Breakdown Analysis
   - Performance Visualization Charts
   - Real-time metrics calculations


In [ ]:
import pandas as pd
import numpy as np
import warnings
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import dash
from dash import dcc, html, dash_table, callback
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc


import socket
# Configuration Variables (Modify as needed)
AOV = 1500  # Average Order Value
CHANNEL_COSTS = {
    'Email': 0.02,
    'Push': 0.01, 
    'SMS': 0.11,
    'WhatsApp': 0.11
}



class CampaignAnalyzer:
    def __init__(self, file_path, aov=AOV, channel_costs=CHANNEL_COSTS):
        self.file_path = file_path
        self.aov = aov
        self.channel_costs = channel_costs
        self.raw_data = None
        self.summary_data = None
        self.channel_data = None
        
    def load_and_process_data(self):
        """Load and process campaign data"""
        try:
            # Load the data
            df = pd.read_csv(self.file_path)
            
            # Basic data cleaning
            df = df.fillna(0)
            
            # Convert numeric columns
            numeric_columns = ['Sent', 'Delivered', 'Unique Impressions', 'Unique Clicks', 
                              'Unique Conversions', 'Unique Click-Through Conversions']
            
            for col in numeric_columns:
                if col in df.columns:
                    df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', ''), errors='coerce').fillna(0)
            
            self.raw_data = df
            return df
        except Exception as e:
            print(f"Error loading data: {e}")
            return None
    
    def calculate_summary_metrics(self):
        """Calculate main summary table metrics"""
        if self.raw_data is None:
            return None
            
        df = self.raw_data
        
        # Group by Journey Name for analysis
        grouped = df.groupby('Journey Name').agg({
            'Campaign ID': 'count',
            'Sent': 'sum',
            'Delivered': 'sum', 
            'Unique Impressions': 'sum',
            'Unique Clicks': 'sum',
            'Unique Conversions': 'sum',
            'Unique Click-Through Conversions': 'sum'
        }).reset_index()
        
        # Rename columns for clarity
        grouped.rename(columns={'Campaign ID': 'Count of Campaign ID'}, inplace=True)
        
        # Calculate performance metrics
        grouped['v_dr'] = np.where(grouped['Sent'] > 0, 
                                  (grouped['Delivered'] / grouped['Sent'] * 100).round(0), 0)
        
        grouped['v_ctr'] = np.where(grouped['Delivered'] > 0,
                                   (grouped['Unique Clicks'] / grouped['Delivered'] * 100).round(1), 0)
        
        grouped['v_cr'] = np.where(grouped['Unique Clicks'] > 0,
                                  (grouped['Unique Click-Through Conversions'] / grouped['Unique Clicks'] * 100).round(1), 0)
        
        grouped['v_ord_per_sent'] = np.where(grouped['Sent'] > 0,
                                            (grouped['Unique Click-Through Conversions'] / grouped['Sent'] * 100).round(2), 0)
        
        # Calculate financial metrics
        financial_metrics = self.calculate_financial_metrics()
        
        # Merge financial data
        summary_df = grouped.merge(financial_metrics[['Journey Name', 'Cost', 'GTV', 'ROI']], 
                                 on='Journey Name', how='left')
        
        # Format the data for display
        summary_df['Row Labels'] = summary_df['Journey Name']
        summary_df['Sum of Sent'] = summary_df['Sent'].apply(lambda x: f"{int(x):,}")
        summary_df['Sum of Delivered'] = summary_df['Delivered'].apply(lambda x: f"{int(x):,}")
        summary_df['Sum of Unique Impressions'] = summary_df['Unique Impressions'].apply(lambda x: f"{int(x):,}")
        summary_df['Sum of Unique Clicks'] = summary_df['Unique Clicks'].apply(lambda x: f"{int(x):,}")
        summary_df['Sum of Unique Click-Through Conversions'] = summary_df['Unique Click-Through Conversions'].apply(lambda x: f"{int(x):,}")
        summary_df['v_dr_formatted'] = summary_df['v_dr'].astype(int).astype(str) + '%'
        summary_df['v_ctr_formatted'] = summary_df['v_ctr'].astype(str) + '%'
        summary_df['v_cr_formatted'] = summary_df['v_cr'].astype(str) + '%'
        summary_df['v_ord_per_sent_formatted'] = summary_df['v_ord_per_sent'].astype(str) + '%'
        summary_df['Cost_formatted'] = summary_df['Cost'].apply(lambda x: f"₹ {x:,.2f}")
        summary_df['GTV_formatted'] = summary_df['GTV'].apply(lambda x: f"{int(x):,}")
        summary_df['ROI_formatted'] = summary_df['ROI'].apply(lambda x: f"₹ {x:.2f}")
        
        self.summary_data = summary_df
        return summary_df
    
    def calculate_financial_metrics(self):
        """Calculate cost, GTV, and ROI metrics"""
        if self.raw_data is None:
            return pd.DataFrame()
            
        df = self.raw_data
        
        financial_df = df.groupby('Journey Name').agg({
            'Sent': 'sum',
            'Channel': lambda x: list(x),
            'Unique Click-Through Conversions': 'sum'
        }).reset_index()
        
        # Calculate costs based on channel
        financial_df['Cost'] = 0
        for idx, row in financial_df.iterrows():
            channel_sent = df[df['Journey Name'] == row['Journey Name']].groupby('Channel')['Sent'].sum()
            total_cost = 0
            for channel, sent_count in channel_sent.items():
                if channel in self.channel_costs:
                    total_cost += sent_count * self.channel_costs[channel]
            financial_df.at[idx, 'Cost'] = total_cost
        
        # Calculate GTV and ROI
        financial_df['GTV'] = financial_df['Unique Click-Through Conversions'] * self.aov
        financial_df['ROI'] = np.where(financial_df['Cost'] > 0,
                                      (financial_df['GTV'] / financial_df['Cost']).round(2), 0)
        
        return financial_df
    
    def calculate_channel_breakdown(self):
        """Calculate channel-wise breakdown"""
        if self.raw_data is None:
            return pd.DataFrame()
            
        df = self.raw_data
        
        # Create channel analysis
        channel_analysis = df.groupby(['Journey Name', 'Channel']).agg({
            'Sent': 'sum',
            'Delivered': 'sum',
            'Unique Click-Through Conversions': 'sum'
        }).reset_index()
        
        # Create a comprehensive breakdown table similar to your reference
        journeys = df['Journey Name'].unique()
        channels = ['Email', 'Push', 'SMS', 'WhatsApp']
        
        # Initialize the breakdown dataframe
        breakdown_data = []
        
        for journey in journeys:
            journey_data = df[df['Journey Name'] == journey]
            row_data = {'Row Labels': journey}
            
            # Calculate totals
            total_sent = journey_data['Sent'].sum()
            total_delivered = journey_data['Delivered'].sum()
            total_conversions = journey_data['Unique Click-Through Conversions'].sum()
            
            # Add channel-wise data
            for channel in channels:
                channel_data = journey_data[journey_data['Channel'] == channel]
                sent = channel_data['Sent'].sum()
                delivered = channel_data['Delivered'].sum()
                conversions = channel_data['Unique Click-Through Conversions'].sum()
                
                row_data[f'{channel}_Sent'] = f"{int(sent):,}" if sent > 0 else ""
                row_data[f'{channel}_Delivered'] = f"{int(delivered):,}" if delivered > 0 else ""
                row_data[f'{channel}_Conversions'] = f"{int(conversions):,}" if conversions > 0 else ""
            
            # Add totals
            row_data['Total_Sent'] = f"{int(total_sent):,}"
            row_data['Total_Delivered'] = f"{int(total_delivered):,}"
            row_data['Total_Conversions'] = f"{int(total_conversions):,}"
            
            breakdown_data.append(row_data)
        
        channel_breakdown = pd.DataFrame(breakdown_data)
        self.channel_data = channel_breakdown
        return channel_breakdown

# Initialize the analyzer
try:
    analyzer = CampaignAnalyzer("report-1750749443470_jBRS6FP_Snapmint _in~~c2ab3517.csv")
    
    # Load and process data
    data_loaded = analyzer.load_and_process_data()
    
    if data_loaded is not None:
        summary_df = analyzer.calculate_summary_metrics()
        channel_df = analyzer.calculate_channel_breakdown()
        
        print(f"✅ Data loaded successfully: {len(data_loaded)} records")
        print(f"📊 Summary table: {len(summary_df)} journeys")
        print(f"📋 Channel breakdown: {len(channel_df)} journeys")
    else:
        print("❌ Failed to load data")
        summary_df = pd.DataFrame()
        channel_df = pd.DataFrame()
        
except Exception as e:
    print(f"❌ Error initializing analyzer: {e}")
    summary_df = pd.DataFrame()
    channel_df = pd.DataFrame()

# Initialize Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define the layout with error handling
def create_layout():
    if summary_df.empty:
        return dbc.Container([
            html.H1("Campaign Performance Analytics Dashboard", className="text-center mb-4 text-danger"),
            html.Hr(),
            dbc.Alert("Failed to load data. Please check your file path and data format.", color="danger")
        ], fluid=True)
    
    return dbc.Container([
        dbc.Row([
            dbc.Col([
                html.H1("Campaign Performance Analytics Dashboard", 
                       className="text-center mb-4 text-primary"),
                html.Hr()
            ])
        ]),
        
        # Key Metrics Row
        dbc.Row([
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H4(f"{summary_df['Sent'].sum():,}", className="text-primary"),
                        html.P("Total Messages Sent", className="mb-0")
                    ])
                ])
            ], width=2),
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H4(f"{summary_df['Delivered'].sum():,}", className="text-success"),
                        html.P("Total Delivered", className="mb-0")
                    ])
                ])
            ], width=2),
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H4(f"{summary_df['Unique Click-Through Conversions'].sum():,}", className="text-info"),
                        html.P("Total Conversions", className="mb-0")
                    ])
                ])
            ], width=2),
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H4(f"₹{summary_df['Cost'].sum():,.2f}", className="text-warning"),
                        html.P("Total Cost", className="mb-0")
                    ])
                ])
            ], width=2),
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H4(f"₹{summary_df['GTV'].sum():,.0f}", className="text-success"),
                        html.P("Total GTV", className="mb-0")
                    ])
                ])
            ], width=2),
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H4(f"{(summary_df['GTV'].sum()/summary_df['Cost'].sum() if summary_df['Cost'].sum() > 0 else 0):.2f}x", className="text-danger"),
                        html.P("Overall ROI", className="mb-0")
                    ])
                ])
            ], width=2),
        ], className="mb-4"),
        
        # Tabs for different views
        dbc.Tabs([
            dbc.Tab(label="Summary Table", tab_id="summary"),
            dbc.Tab(label="Channel Breakdown", tab_id="channels"),
            dbc.Tab(label="Performance Charts", tab_id="charts"),
        ], id="tabs", active_tab="summary"),
        
        html.Div(id="tab-content", className="mt-4")
    ], fluid=True)

app.layout = create_layout()

@app.callback(
    Output("tab-content", "children"),
    Input("tabs", "active_tab")
)
def render_tab_content(active_tab):
    if summary_df.empty:
        return dbc.Alert("No data available to display.", color="warning")
    
    try:
        if active_tab == "summary":
            # Create summary table
            summary_display_df = summary_df[[
                'Row Labels', 'Count of Campaign ID', 'Sum of Sent', 'Sum of Delivered',
                'Sum of Unique Impressions', 'Sum of Unique Clicks', 
                'Sum of Unique Click-Through Conversions', 'v_dr_formatted', 
                'v_ctr_formatted', 'v_cr_formatted', 'v_ord_per_sent_formatted',
                'Cost_formatted', 'GTV_formatted', 'ROI_formatted'
            ]].copy()
            
            summary_display_df.columns = [
                'Row Labels', 'Count of Campaign ID', 'Sum of Sent', 'Sum of Delivered',
                'Sum of Unique Impressions', 'Sum of Unique Clicks', 
                'Sum of Unique Click-Through Conversions', 'v_dr', 'v_ctr', 'v_cr',
                'v_ord_per_sent', 'Cost', 'GTV', 'ROI'
            ]
            
            return dash_table.DataTable(
                data=summary_display_df.to_dict('records'),
                columns=[{"name": i, "id": i} for i in summary_display_df.columns],
                style_table={'overflowX': 'auto'},
                style_cell={
                    'textAlign': 'left',
                    'padding': '10px',
                    'fontFamily': 'Arial',
                    'fontSize': '12px'
                },
                style_header={
                    'backgroundColor': 'rgb(230, 230, 230)',
                    'fontWeight': 'bold',
                    'fontSize': '14px'
                },
                style_data_conditional=[
                    {
                        'if': {'row_index': 'odd'},
                        'backgroundColor': 'rgb(248, 248, 248)'
                    }
                ],
                sort_action="native",
                filter_action="native",
                page_size=20,
                export_format="xlsx",
                export_headers="display"
            )
        
        elif active_tab == "channels":
            if channel_df.empty:
                return dbc.Alert("No channel breakdown data available.", color="warning")
                
            return dash_table.DataTable(
                data=channel_df.to_dict('records'),
                columns=[{"name": i, "id": i} for i in channel_df.columns],
                style_table={'overflowX': 'auto'},
                style_cell={
                    'textAlign': 'left',
                    'padding': '8px',
                    'fontFamily': 'Arial',
                    'fontSize': '11px',
                    'minWidth': '80px'
                },
                style_header={
                    'backgroundColor': 'rgb(230, 230, 230)',
                    'fontWeight': 'bold',
                    'fontSize': '12px'
                },
                style_data_conditional=[
                    {
                        'if': {'row_index': 'odd'},
                        'backgroundColor': 'rgb(248, 248, 248)'
                    }
                ],
                sort_action="native",
                filter_action="native",
                page_size=15,
                export_format="xlsx",
                export_headers="display"
            )
        
        elif active_tab == "charts":
            # Create performance charts with error handling
            try:
                # Filter data for meaningful charts
                chart_data = summary_df[summary_df['ROI'] > 0].nlargest(10, 'ROI')
                
                if chart_data.empty:
                    return dbc.Alert("No data available for charts.", color="warning")
                
                # Chart 1: Top ROI Performance
                fig1 = px.bar(chart_data, 
                             x='Journey Name', y='ROI', 
                             title="Top 10 Journeys by ROI",
                             labels={'ROI': 'Return on Investment'})
                fig1.update_layout(xaxis_tickangle=45, height=400)
                
                # Chart 2: Cost vs GTV Analysis
                scatter_data = summary_df[(summary_df['Cost'] > 0) & (summary_df['GTV'] > 0)]
                fig2 = px.scatter(scatter_data, 
                                 x='Cost', y='GTV', 
                                 size='Unique Click-Through Conversions',
                                 hover_name='Journey Name',
                                 title="Cost vs GTV Analysis",
                                 labels={'Cost': 'Total Cost (₹)', 'GTV': 'Gross Transaction Value (₹)'})
                fig2.update_layout(height=400)
                
                # Chart 3: Volume Distribution
                volume_data = summary_df.nlargest(8, 'Sent')
                fig3 = px.pie(volume_data, 
                             values='Sent', names='Journey Name',
                             title="Top 8 Journeys by Messages Sent")
                fig3.update_layout(height=400)
                
                # Chart 4: Channel Performance
                if not channel_df.empty:
                    # Calculate channel totals
                    channel_totals = {}
                    for channel in ['Email', 'Push', 'SMS', 'WhatsApp']:
                        total = 0
                        for _, row in channel_df.iterrows():
                            val = row.get(f'{channel}_Sent', '0')
                            if val and val != '':
                                total += int(val.replace(',', ''))
                        channel_totals[channel] = total
                    
                    fig4 = px.bar(x=list(channel_totals.keys()), 
                                 y=list(channel_totals.values()),
                                 title="Messages Sent by Channel",
                                 labels={'x': 'Channel', 'y': 'Total Messages Sent'})
                    fig4.update_layout(height=400)
                else:
                    fig4 = px.bar(title="Channel data not available")
                
                return dbc.Row([
                    dbc.Col([dcc.Graph(figure=fig1)], width=6),
                    dbc.Col([dcc.Graph(figure=fig2)], width=6),
                    dbc.Col([dcc.Graph(figure=fig3)], width=6),
                    dbc.Col([dcc.Graph(figure=fig4)], width=6)
                ])
                
            except Exception as e:
                return dbc.Alert(f"Error creating charts: {str(e)}", color="danger")
    
    except Exception as e:
        return dbc.Alert(f"Error rendering content: {str(e)}", color="danger")
    
def get_local_ip():
    """Get the local IP address of the machine"""
    try:
        # Connect to a remote server to determine local IP
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        s.connect(("8.8.8.8", 80))
        local_ip = s.getsockname()[0]
        s.close()
        return local_ip
    except Exception:
        return "localhost"
warnings.filterwarnings('ignore')


if __name__ == "__main__":
    local_ip = get_local_ip()
    
    print("🚀 Starting Campaign Analytics Dashboard...")
    print("📊 Dashboard URLs:")
    print(f"   - Local Access: http://127.0.0.1:8052")
    print(f"   - Network Access: http://{local_ip}:8052")
    print(f"   - WiFi Share URL: http://{local_ip}:8052")
    print("\n📋 Features available:")
    print("   - Interactive Summary Table (sortable, filterable, exportable)")
    print("   - Channel Breakdown Analysis")
    print("   - Performance Visualization Charts")
    print("   - Real-time metrics calculations")
    print(f"\n🌐 Share this URL with others on your WiFi:")
    print(f"   http://{local_ip}:8052")
    print("\n" + "="*60)
    
    try:
        # Use run_server instead of run, and specify the correct parameters
        app.run(
            debug=False,  # Set to False for network access
            host='0.0.0.0',  # Listen on all interfaces
            port=8052,  # Your chosen port
            dev_tools_hot_reload=False  # Disable hot reload for network access
        )
    except Exception as e:
        print(f"❌ Error starting server: {e}")
        print("📝 Trying alternative configuration...")
        try:
            # Fallback: Run on local IP only
            app.run(
                debug=False,
                host=local_ip,
                port=8052
            )
        except Exception as e2:
            print(f"❌ Fallback failed: {e2}")
            print("🔧 Try running on localhost only:")
            app.run(debug=True, host='127.0.0.1', port=8052)

In [1]:
import pandas as pd
import numpy as np
import warnings
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import dash
from dash import dcc, html, dash_table, callback
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
import socket

warnings.filterwarnings('ignore')

# Configuration Variables (Modify as needed)
AOV = 1500  # Average Order Value
CHANNEL_COSTS = {
    'Email': 0.02,
    'Push': 0.01, 
    'SMS': 0.11,
    'WhatsApp': 0.11
}

class CampaignAnalyzer:
    def __init__(self, file_path, aov=AOV, channel_costs=CHANNEL_COSTS):
        self.file_path = file_path
        self.aov = aov
        self.channel_costs = channel_costs
        self.raw_data = None
        self.summary_data = None
        self.channel_data = None
        
    def load_and_process_data(self):
        """Load and process campaign data"""
        try:
            # Load the data
            df = pd.read_csv(self.file_path)
            
            # Basic data cleaning
            df = df.fillna(0)
            
            # Convert numeric columns with better error handling
            numeric_columns = ['Sent', 'Delivered', 'Unique Impressions', 'Unique Clicks', 
                              'Unique Conversions', 'Unique Click-Through Conversions']
            
            for col in numeric_columns:
                if col in df.columns:
                    # Convert to string first, replace commas, then convert to numeric
                    df[col] = pd.to_numeric(
                        df[col].astype(str).str.replace(',', '').str.replace('nan', '0'), 
                        errors='coerce'
                    ).fillna(0)
            
            # Convert Day column to datetime if available with better error handling
            if 'Day' in df.columns:
                df['Day'] = pd.to_datetime(df['Day'], errors='coerce')
            
            # Ensure Journey Name is string type to avoid sorting issues
            if 'Journey Name' in df.columns:
                df['Journey Name'] = df['Journey Name'].astype(str)
            
            # Ensure Channel is string type
            if 'Channel' in df.columns:
                df['Channel'] = df['Channel'].astype(str)
            
            # Ensure Status is string type
            if 'Status' in df.columns:
                df['Status'] = df['Status'].astype(str)
                
            self.raw_data = df
            return df
        except Exception as e:
            print(f"Error loading data: {e}")
            return None
    
    def calculate_summary_metrics(self, filtered_df=None):
        """Calculate main summary table metrics"""
        df = filtered_df if filtered_df is not None else self.raw_data
        
        if df is None or df.empty:
            return pd.DataFrame()
            
        try:
            # Group by Journey Name for analysis
            grouped = df.groupby('Journey Name').agg({
                'Campaign ID': 'count',
                'Sent': 'sum',
                'Delivered': 'sum', 
                'Unique Impressions': 'sum',
                'Unique Clicks': 'sum',
                'Unique Conversions': 'sum',
                'Unique Click-Through Conversions': 'sum'
            }).reset_index()
            
            # Rename columns for clarity
            grouped.rename(columns={'Campaign ID': 'Count of Campaign ID'}, inplace=True)
            
            # Calculate performance metrics with safe division
            grouped['v_dr'] = np.where(grouped['Sent'] > 0, 
                                      (grouped['Delivered'] / grouped['Sent'] * 100).round(0), 0)
            
            grouped['v_ctr'] = np.where(grouped['Delivered'] > 0,
                                       (grouped['Unique Clicks'] / grouped['Delivered'] * 100).round(1), 0)
            
            grouped['v_cr'] = np.where(grouped['Unique Clicks'] > 0,
                                      (grouped['Unique Click-Through Conversions'] / grouped['Unique Clicks'] * 100).round(1), 0)
            
            grouped['v_ord_per_sent'] = np.where(grouped['Sent'] > 0,
                                                (grouped['Unique Click-Through Conversions'] / grouped['Sent'] * 100).round(2), 0)
            
            # Calculate financial metrics
            financial_metrics = self.calculate_financial_metrics(filtered_df)
            
            if not financial_metrics.empty:
                # Merge financial data
                summary_df = grouped.merge(financial_metrics[['Journey Name', 'Cost', 'GTV', 'ROI']], 
                                         on='Journey Name', how='left')
            else:
                # Add empty financial columns if calculation fails
                summary_df = grouped.copy()
                summary_df['Cost'] = 0
                summary_df['GTV'] = 0
                summary_df['ROI'] = 0
            
            # Format the data for display
            summary_df['Row Labels'] = summary_df['Journey Name']
            summary_df['Sum of Sent'] = summary_df['Sent'].apply(lambda x: f"{int(x):,}")
            summary_df['Sum of Delivered'] = summary_df['Delivered'].apply(lambda x: f"{int(x):,}")
            summary_df['Sum of Unique Impressions'] = summary_df['Unique Impressions'].apply(lambda x: f"{int(x):,}")
            summary_df['Sum of Unique Clicks'] = summary_df['Unique Clicks'].apply(lambda x: f"{int(x):,}")
            summary_df['Sum of Unique Click-Through Conversions'] = summary_df['Unique Click-Through Conversions'].apply(lambda x: f"{int(x):,}")
            summary_df['v_dr_formatted'] = summary_df['v_dr'].astype(int).astype(str) + '%'
            summary_df['v_ctr_formatted'] = summary_df['v_ctr'].astype(str) + '%'
            summary_df['v_cr_formatted'] = summary_df['v_cr'].astype(str) + '%'
            summary_df['v_ord_per_sent_formatted'] = summary_df['v_ord_per_sent'].astype(str) + '%'
            summary_df['Cost_formatted'] = summary_df['Cost'].apply(lambda x: f"₹ {x:,.2f}")
            summary_df['GTV_formatted'] = summary_df['GTV'].apply(lambda x: f"{int(x):,}")
            summary_df['ROI_formatted'] = summary_df['ROI'].apply(lambda x: f"₹ {x:.2f}")
            
            return summary_df
            
        except Exception as e:
            print(f"Error in calculate_summary_metrics: {e}")
            return pd.DataFrame()
    
    def calculate_financial_metrics(self, filtered_df=None):
        """Calculate cost, GTV, and ROI metrics"""
        df = filtered_df if filtered_df is not None else self.raw_data
        
        if df is None or df.empty:
            return pd.DataFrame()
        
        try:
            financial_df = df.groupby('Journey Name').agg({
                'Sent': 'sum',
                'Channel': lambda x: list(x),
                'Unique Click-Through Conversions': 'sum'
            }).reset_index()
            
            # Calculate costs based on channel
            financial_df['Cost'] = 0
            for idx, row in financial_df.iterrows():
                try:
                    channel_sent = df[df['Journey Name'] == row['Journey Name']].groupby('Channel')['Sent'].sum()
                    total_cost = 0
                    for channel, sent_count in channel_sent.items():
                        if channel in self.channel_costs:
                            total_cost += sent_count * self.channel_costs[channel]
                    financial_df.at[idx, 'Cost'] = total_cost
                except Exception as e:
                    print(f"Error calculating cost for {row['Journey Name']}: {e}")
                    financial_df.at[idx, 'Cost'] = 0
            
            # Calculate GTV and ROI
            financial_df['GTV'] = financial_df['Unique Click-Through Conversions'] * self.aov
            financial_df['ROI'] = np.where(financial_df['Cost'] > 0,
                                          (financial_df['GTV'] / financial_df['Cost']).round(2), 0)
            
            return financial_df
            
        except Exception as e:
            print(f"Error in calculate_financial_metrics: {e}")
            return pd.DataFrame()
    
    def calculate_channel_breakdown(self, filtered_df=None):
        """Calculate channel-wise breakdown"""
        df = filtered_df if filtered_df is not None else self.raw_data
        
        if df is None or df.empty:
            return pd.DataFrame()
        
        try:
            # Create channel analysis
            channel_analysis = df.groupby(['Journey Name', 'Channel']).agg({
                'Sent': 'sum',
                'Delivered': 'sum',
                'Unique Click-Through Conversions': 'sum'
            }).reset_index()
            
            # Create a comprehensive breakdown table
            journeys = df['Journey Name'].unique()
            channels = ['Email', 'Push', 'SMS', 'WhatsApp']
            
            # Initialize the breakdown dataframe
            breakdown_data = []
            
            for journey in journeys:
                journey_data = df[df['Journey Name'] == journey]
                row_data = {'Row Labels': str(journey)}
                
                # Calculate totals
                total_sent = journey_data['Sent'].sum()
                total_delivered = journey_data['Delivered'].sum()
                total_conversions = journey_data['Unique Click-Through Conversions'].sum()
                
                # Add channel-wise data
                for channel in channels:
                    channel_data = journey_data[journey_data['Channel'] == channel]
                    sent = channel_data['Sent'].sum()
                    delivered = channel_data['Delivered'].sum()
                    conversions = channel_data['Unique Click-Through Conversions'].sum()
                    
                    row_data[f'{channel}_Sent'] = f"{int(sent):,}" if sent > 0 else ""
                    row_data[f'{channel}_Delivered'] = f"{int(delivered):,}" if delivered > 0 else ""
                    row_data[f'{channel}_Conversions'] = f"{int(conversions):,}" if conversions > 0 else ""
                
                # Add totals
                row_data['Total_Sent'] = f"{int(total_sent):,}"
                row_data['Total_Delivered'] = f"{int(total_delivered):,}"
                row_data['Total_Conversions'] = f"{int(total_conversions):,}"
                
                breakdown_data.append(row_data)
            
            channel_breakdown = pd.DataFrame(breakdown_data)
            return channel_breakdown
            
        except Exception as e:
            print(f"Error in calculate_channel_breakdown: {e}")
            return pd.DataFrame()

# Initialize the analyzer with better error handling
try:
    analyzer = CampaignAnalyzer("report-1750749443470_jBRS6FP_Snapmint _in~~c2ab3517.csv")
    
    # Load and process data
    data_loaded = analyzer.load_and_process_data()
    
    if data_loaded is not None:
        summary_df = analyzer.calculate_summary_metrics()
        channel_df = analyzer.calculate_channel_breakdown()
        
        print(f"✅ Data loaded successfully: {len(data_loaded)} records")
        print(f"📊 Summary table: {len(summary_df)} journeys")
        print(f"📋 Channel breakdown: {len(channel_df)} journeys")
        
        # Get unique values for filters with safe type conversion
        try:
            unique_journeys = sorted([str(j) for j in data_loaded['Journey Name'].unique() if pd.notna(j)])
            unique_channels = sorted([str(c) for c in data_loaded['Channel'].unique() if pd.notna(c)])
            unique_statuses = sorted([str(s) for s in data_loaded['Status'].unique() if pd.notna(s)]) if 'Status' in data_loaded.columns else []
            
            # Date range
            if 'Day' in data_loaded.columns:
                valid_dates = data_loaded['Day'].dropna()
                if not valid_dates.empty:
                    min_date = valid_dates.min()
                    max_date = valid_dates.max()
                else:
                    min_date = max_date = None
            else:
                min_date = max_date = None
                
        except Exception as e:
            print(f"Error processing filter options: {e}")
            unique_journeys = []
            unique_channels = []
            unique_statuses = []
            min_date = max_date = None
            
    else:
        print("❌ Failed to load data")
        summary_df = pd.DataFrame()
        channel_df = pd.DataFrame()
        unique_journeys = []
        unique_channels = []
        unique_statuses = []
        min_date = max_date = None
        
except Exception as e:
    print(f"❌ Error initializing analyzer: {e}")
    summary_df = pd.DataFrame()
    channel_df = pd.DataFrame()
    unique_journeys = []
    unique_channels = []
    unique_statuses = []
    min_date = max_date = None

# Initialize Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

def get_local_ip():
    """Get the local IP address of the machine"""
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        s.connect(("8.8.8.8", 80))
        local_ip = s.getsockname()[0]
        s.close()
        return local_ip
    except Exception:
        return "127.0.0.1"

# Define the layout with filters
def create_layout():
    if summary_df.empty:
        return dbc.Container([
            html.H1("Campaign Performance Analytics Dashboard", className="text-center mb-4 text-danger"),
            html.Hr(),
            dbc.Alert("Failed to load data. Please check your file path and data format.", color="danger")
        ], fluid=True)
    
    return dbc.Container([
        dbc.Row([
            dbc.Col([
                html.H1("Campaign Performance Analytics Dashboard", 
                       className="text-center mb-4 text-primary"),
                html.Hr()
            ])
        ]),
        
        # Filter Controls Section
        dbc.Card([
            dbc.CardHeader([
                html.H4("🔍 Filter Controls", className="mb-0")
            ]),
            dbc.CardBody([
                dbc.Row([
                    # Journey Selection
                    dbc.Col([
                        html.Label("📋 Select Journeys:", className="fw-bold"),
                        html.Div([
                            dbc.Button("Select All", id="select-all-journeys", size="sm", color="outline-primary", className="me-2 mb-2"),
                            dbc.Button("Clear All", id="clear-all-journeys", size="sm", color="outline-secondary", className="me-2 mb-2"),
                            dbc.Button("Top 10 ROI", id="top-roi-journeys", size="sm", color="outline-success", className="mb-2"),
                        ]),
                        dcc.Checklist(
                            id='journey-checklist',
                            options=[{'label': journey, 'value': journey} for journey in unique_journeys],
                            value=unique_journeys[:10] if len(unique_journeys) > 10 else unique_journeys,  # Limit initial selection
                            style={'maxHeight': '200px', 'overflowY': 'scroll', 'border': '1px solid #ccc', 'padding': '10px'},
                            className="mt-2"
                        )
                    ], width=4),
                    
                    # Channel Selection  
                    dbc.Col([
                        html.Label("📡 Select Channels:", className="fw-bold"),
                        html.Div([
                            dbc.Button("Select All", id="select-all-channels", size="sm", color="outline-primary", className="me-2 mb-2"),
                            dbc.Button("Clear All", id="clear-all-channels", size="sm", color="outline-secondary", className="mb-2"),
                        ]),
                        dcc.Checklist(
                            id='channel-checklist',
                            options=[{'label': channel, 'value': channel} for channel in unique_channels],
                            value=unique_channels,  # All selected by default
                            className="mt-2"
                        )
                    ], width=2),
                    
                    # Status Selection
                    dbc.Col([
                        html.Label("📊 Campaign Status:", className="fw-bold"),
                        dcc.Checklist(
                            id='status-checklist',
                            options=[{'label': status, 'value': status} for status in unique_statuses],
                            value=unique_statuses,  # All selected by default
                            className="mt-2"
                        )
                    ], width=2),
                    
                    # Date Range and Performance Filters
                    dbc.Col([
                        html.Label("📅 Date Range:", className="fw-bold"),
                        dcc.DatePickerRange(
                            id='date-picker-range',
                            start_date=min_date,
                            end_date=max_date,
                            display_format='YYYY-MM-DD',
                            className="mb-3"
                        ) if min_date and max_date else html.P("No date data available"),
                        
                        html.Label("🎯 Performance Filter:", className="fw-bold"),
                        dcc.Dropdown(
                            id='performance-filter',
                            options=[
                                {'label': 'All Campaigns', 'value': 'all'},
                                {'label': 'High ROI (>10x)', 'value': 'high_roi'},
                                {'label': 'High CTR (>2%)', 'value': 'high_ctr'},
                                {'label': 'High Conversion (>1%)', 'value': 'high_conv'},
                                {'label': 'Low Performers', 'value': 'low_perf'}
                            ],
                            value='all',
                            className="mt-2"
                        )
                    ], width=4),
                ]),
                
                html.Hr(),
                
                # Quick Filter Buttons
                dbc.Row([
                    dbc.Col([
                        html.Label("⚡ Quick Filters:", className="fw-bold"),
                        html.Div([
                            dbc.Button("🏆 Top Performers", id="quick-top-performers", size="sm", color="success", className="me-2 mb-2"),
                            dbc.Button("📧 Email Only", id="quick-email-only", size="sm", color="info", className="me-2 mb-2"),
                            dbc.Button("📱 Push Only", id="quick-push-only", size="sm", color="warning", className="me-2 mb-2"),
                            dbc.Button("💰 High Value", id="quick-high-value", size="sm", color="primary", className="me-2 mb-2"),
                            dbc.Button("🔄 Reset All", id="reset-filters", size="sm", color="secondary", className="mb-2"),
                        ])
                    ])
                ])
            ])
        ], className="mb-4"),
        
        # Key Metrics Row (Dynamic based on filters)
        html.Div(id="metrics-cards"),
        
        # Tabs for different views
        dbc.Tabs([
            dbc.Tab(label="Summary Table", tab_id="summary"),
            dbc.Tab(label="Channel Breakdown", tab_id="channels"),
            dbc.Tab(label="Performance Charts", tab_id="charts"),
            dbc.Tab(label="Campaign Details", tab_id="details"),
        ], id="tabs", active_tab="summary"),
        
        html.Div(id="tab-content", className="mt-4")
    ], fluid=True)

app.layout = create_layout()

# Callback for filter interactions with better error handling
@app.callback(
    [Output('journey-checklist', 'value'),
     Output('channel-checklist', 'value')],
    [Input('select-all-journeys', 'n_clicks'),
     Input('clear-all-journeys', 'n_clicks'),
     Input('top-roi-journeys', 'n_clicks'),
     Input('select-all-channels', 'n_clicks'),
     Input('clear-all-channels', 'n_clicks'),
     Input('quick-top-performers', 'n_clicks'),
     Input('quick-email-only', 'n_clicks'),
     Input('quick-push-only', 'n_clicks'),
     Input('quick-high-value', 'n_clicks'),
     Input('reset-filters', 'n_clicks')],
    [State('journey-checklist', 'value'),
     State('channel-checklist', 'value')],
    prevent_initial_call=True
)
def update_filters(select_all_j, clear_all_j, top_roi_j, select_all_c, clear_all_c,
                  quick_top, quick_email, quick_push, quick_value, reset_all,
                  current_journeys, current_channels):
    
    try:
        ctx = dash.callback_context
        if not ctx.triggered:
            return current_journeys or [], current_channels or []
        
        button_id = ctx.triggered[0]['prop_id'].split('.')[0]
        
        if button_id == 'select-all-journeys':
            return unique_journeys, current_channels or []
        elif button_id == 'clear-all-journeys':
            return [], current_channels or []
        elif button_id == 'top-roi-journeys':
            # Get top 10 ROI journeys safely
            if not summary_df.empty and 'ROI' in summary_df.columns:
                try:
                    top_journeys = summary_df.nlargest(10, 'ROI')['Journey Name'].tolist()
                    return top_journeys, current_channels or []
                except:
                    pass
            return current_journeys or [], current_channels or []
        elif button_id == 'select-all-channels':
            return current_journeys or [], unique_channels
        elif button_id == 'clear-all-channels':
            return current_journeys or [], []
        elif button_id == 'quick-top-performers':
            if not summary_df.empty and 'ROI' in summary_df.columns:
                try:
                    top_journeys = summary_df.nlargest(5, 'ROI')['Journey Name'].tolist()
                    return top_journeys, current_channels or []
                except:
                    pass
            return current_journeys or [], current_channels or []
        elif button_id == 'quick-email-only':
            return current_journeys or [], ['Email']
        elif button_id == 'quick-push-only':
            return current_journeys or [], ['Push']
        elif button_id == 'quick-high-value':
            if not summary_df.empty and 'GTV' in summary_df.columns:
                try:
                    median_gtv = summary_df['GTV'].median()
                    high_value = summary_df[summary_df['GTV'] > median_gtv]['Journey Name'].tolist()
                    return high_value, current_channels or []
                except:
                    pass
            return current_journeys or [], current_channels or []
        elif button_id == 'reset-filters':
            return unique_journeys, unique_channels
        
        return current_journeys or [], current_channels or []
        
    except Exception as e:
        print(f"Error in update_filters: {e}")
        return current_journeys or [], current_channels or []

# Dynamic metrics cards callback with better error handling
@app.callback(
    Output("metrics-cards", "children"),
    [Input('journey-checklist', 'value'),
     Input('channel-checklist', 'value'),
     Input('status-checklist', 'value'),
     Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date'),
     Input('performance-filter', 'value')]
)
def update_metrics_cards(selected_journeys, selected_channels, selected_statuses, start_date, end_date, perf_filter):
    try:
        if data_loaded.empty:
            return dbc.Alert("No data available", color="warning", className="mb-4")
        
        # Apply filters safely
        filtered_data = data_loaded.copy()
        
        if selected_journeys:
            filtered_data = filtered_data[filtered_data['Journey Name'].isin(selected_journeys)]
        if selected_channels:
            filtered_data = filtered_data[filtered_data['Channel'].isin(selected_channels)]
        if selected_statuses:
            filtered_data = filtered_data[filtered_data['Status'].isin(selected_statuses)]
        if start_date and end_date and 'Day' in filtered_data.columns:
            filtered_data = filtered_data[(filtered_data['Day'] >= start_date) & (filtered_data['Day'] <= end_date)]
        
        # Calculate metrics for filtered data
        filtered_summary = analyzer.calculate_summary_metrics(filtered_data)
        
        if filtered_summary.empty:
            return dbc.Alert("No data matches the selected filters.", color="warning", className="mb-4")
        
        # Create metrics cards safely
        total_sent = filtered_summary['Sent'].sum() if 'Sent' in filtered_summary.columns else 0
        total_delivered = filtered_summary['Delivered'].sum() if 'Delivered' in filtered_summary.columns else 0
        total_conversions = filtered_summary['Unique Click-Through Conversions'].sum() if 'Unique Click-Through Conversions' in filtered_summary.columns else 0
        total_cost = filtered_summary['Cost'].sum() if 'Cost' in filtered_summary.columns else 0
        total_gtv = filtered_summary['GTV'].sum() if 'GTV' in filtered_summary.columns else 0
        overall_roi = total_gtv / total_cost if total_cost > 0 else 0
        
        return dbc.Row([
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H4(f"{total_sent:,}", className="text-primary"),
                        html.P("Total Messages Sent", className="mb-0"),
                        html.Small(f"📊 {len(filtered_summary)} journeys", className="text-muted")
                    ])
                ])
            ], width=2),
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H4(f"{total_delivered:,}", className="text-success"),
                        html.P("Total Delivered", className="mb-0"),
                        html.Small(f"📈 {(total_delivered/total_sent*100 if total_sent > 0 else 0):.1f}% delivery rate", className="text-muted")
                    ])
                ])
            ], width=2),
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H4(f"{total_conversions:,}", className="text-info"),
                        html.P("Total Conversions", className="mb-0"),
                        html.Small(f"🎯 {(total_conversions/total_delivered*100 if total_delivered > 0 else 0):.2f}% conv rate", className="text-muted")
                    ])
                ])
            ], width=2),
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H4(f"₹{total_cost:,.0f}", className="text-warning"),
                        html.P("Total Cost", className="mb-0"),
                        html.Small(f"💰 ₹{(total_cost/total_sent if total_sent > 0 else 0):.3f} per message", className="text-muted")
                    ])
                ])
            ], width=2),
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H4(f"₹{total_gtv:,.0f}", className="text-success"),
                        html.P("Total GTV", className="mb-0"),
                        html.Small(f"💎 ₹{(total_gtv/total_conversions if total_conversions > 0 else 0):,.0f} per conversion", className="text-muted")
                    ])
                ])
            ], width=2),
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H4(f"{overall_roi:.2f}x", className="text-danger"),
                        html.P("Overall ROI", className="mb-0"),
                        html.Small(f"🚀 {'Profitable' if overall_roi > 1 else 'Loss'}", className="text-muted")
                    ])
                ])
            ], width=2),
        ], className="mb-4")
        
    except Exception as e:
        print(f"Error in update_metrics_cards: {e}")
        return dbc.Alert("Error calculating metrics", color="warning", className="mb-4")

# Main content callback with filters and better error handling
@app.callback(
    Output("tab-content", "children"),
    [Input("tabs", "active_tab"),
     Input('journey-checklist', 'value'),
     Input('channel-checklist', 'value'),
     Input('status-checklist', 'value'),
     Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date'),
     Input('performance-filter', 'value')]
)
def render_tab_content(active_tab, selected_journeys, selected_channels, selected_statuses, start_date, end_date, perf_filter):
    try:
        if data_loaded.empty:
            return dbc.Alert("No data available to display.", color="warning")
        
        # Apply filters safely
        filtered_data = data_loaded.copy()
        
        if selected_journeys:
            filtered_data = filtered_data[filtered_data['Journey Name'].isin(selected_journeys)]
        if selected_channels:
            filtered_data = filtered_data[filtered_data['Channel'].isin(selected_channels)]
        if selected_statuses:
            filtered_data = filtered_data[filtered_data['Status'].isin(selected_statuses)]
        if start_date and end_date and 'Day' in filtered_data.columns:
            filtered_data = filtered_data[(filtered_data['Day'] >= start_date) & (filtered_data['Day'] <= end_date)]
        
        # Apply performance filter
        filtered_summary = analyzer.calculate_summary_metrics(filtered_data)
        
        if perf_filter == 'high_roi' and 'ROI' in filtered_summary.columns:
            filtered_summary = filtered_summary[filtered_summary['ROI'] > 10]
        elif perf_filter == 'high_ctr' and 'v_ctr' in filtered_summary.columns:
            filtered_summary = filtered_summary[filtered_summary['v_ctr'] > 2]
        elif perf_filter == 'high_conv' and 'v_ord_per_sent' in filtered_summary.columns:
            filtered_summary = filtered_summary[filtered_summary['v_ord_per_sent'] > 1]
        elif perf_filter == 'low_perf' and 'ROI' in filtered_summary.columns:
            filtered_summary = filtered_summary[filtered_summary['ROI'] < 5]
        
        if filtered_summary.empty:
            return dbc.Alert("No data matches the selected filters.", color="warning")
        
        if active_tab == "summary":
            # Create summary table with safe column selection
            available_cols = [
                'Row Labels', 'Count of Campaign ID', 'Sum of Sent', 'Sum of Delivered',
                'Sum of Unique Impressions', 'Sum of Unique Clicks', 
                'Sum of Unique Click-Through Conversions', 'v_dr_formatted', 
                'v_ctr_formatted', 'v_cr_formatted', 'v_ord_per_sent_formatted',
                'Cost_formatted', 'GTV_formatted', 'ROI_formatted'
            ]
            
            # Only include columns that exist
            display_cols = [col for col in available_cols if col in filtered_summary.columns]
            summary_display_df = filtered_summary[display_cols].copy()
            
            # Rename columns for display
            column_names = {
                'Row Labels': 'Row Labels',
                'Count of Campaign ID': 'Count of Campaign ID',
                'Sum of Sent': 'Sum of Sent',
                'Sum of Delivered': 'Sum of Delivered',
                'Sum of Unique Impressions': 'Sum of Unique Impressions',
                'Sum of Unique Clicks': 'Sum of Unique Clicks',
                'Sum of Unique Click-Through Conversions': 'Sum of Unique Click-Through Conversions',
                'v_dr_formatted': 'v_dr',
                'v_ctr_formatted': 'v_ctr',
                'v_cr_formatted': 'v_cr',
                'v_ord_per_sent_formatted': 'v_ord_per_sent',
                'Cost_formatted': 'Cost',
                'GTV_formatted': 'GTV',
                'ROI_formatted': 'ROI'
            }
            
            summary_display_df.columns = [column_names.get(col, col) for col in summary_display_df.columns]
            
            return [
                html.H5(f"📊 Showing {len(summary_display_df)} journeys (filtered from {len(summary_df)} total)", className="mb-3"),
                dash_table.DataTable(
                    data=summary_display_df.to_dict('records'),
                    columns=[{"name": i, "id": i} for i in summary_display_df.columns],
                    style_table={'overflowX': 'auto'},
                    style_cell={
                        'textAlign': 'left',
                        'padding': '10px',
                        'fontFamily': 'Arial',
                        'fontSize': '12px'
                    },
                    style_header={
                        'backgroundColor': 'rgb(230, 230, 230)',
                        'fontWeight': 'bold',
                        'fontSize': '14px'
                    },
                    style_data_conditional=[
                        {
                            'if': {'row_index': 'odd'},
                            'backgroundColor': 'rgb(248, 248, 248)'
                        }
                    ],
                    sort_action="native",
                    filter_action="native",
                    page_size=20,
                    export_format="xlsx",
                    export_headers="display"
                )
            ]
        
        elif active_tab == "channels":
            filtered_channel_df = analyzer.calculate_channel_breakdown(filtered_data)
            
            if filtered_channel_df.empty:
                return dbc.Alert("No channel breakdown data available for selected filters.", color="warning")
                
            return [
                html.H5(f"📡 Channel breakdown for {len(filtered_channel_df)} journeys", className="mb-3"),
                dash_table.DataTable(
                    data=filtered_channel_df.to_dict('records'),
                    columns=[{"name": i, "id": i} for i in filtered_channel_df.columns],
                    style_table={'overflowX': 'auto'},
                    style_cell={
                        'textAlign': 'left',
                        'padding': '8px',
                        'fontFamily': 'Arial',
                        'fontSize': '11px',
                        'minWidth': '80px'
                    },
                    style_header={
                        'backgroundColor': 'rgb(230, 230, 230)',
                        'fontWeight': 'bold',
                        'fontSize': '12px'
                    },
                    style_data_conditional=[
                        {
                            'if': {'row_index': 'odd'},
                            'backgroundColor': 'rgb(248, 248, 248)'
                        }
                    ],
                    sort_action="native",
                    filter_action="native",
                    page_size=15,
                    export_format="xlsx",
                    export_headers="display"
                )
            ]
        
        elif active_tab == "charts":
            # Create performance charts with filtered data
            try:
                if 'ROI' in filtered_summary.columns:
                    chart_data = filtered_summary[filtered_summary['ROI'] > 0].nlargest(10, 'ROI')
                else:
                    chart_data = filtered_summary.head(10)
                
                if chart_data.empty:
                    return dbc.Alert("No data available for charts with current filters.", color="warning")
                
                # Chart 1: Top ROI Performance
                if 'ROI' in chart_data.columns:
                    fig1 = px.bar(chart_data, 
                                 x='Journey Name', y='ROI', 
                                 title=f"Top {len(chart_data)} Journeys by ROI (Filtered)",
                                 labels={'ROI': 'Return on Investment'})
                else:
                    fig1 = px.bar(chart_data, 
                                 x='Journey Name', y='Sent', 
                                 title=f"Top {len(chart_data)} Journeys by Messages Sent (Filtered)")
                fig1.update_layout(xaxis_tickangle=45, height=400)
                
                # Chart 2: Cost vs GTV Analysis
                if 'Cost' in filtered_summary.columns and 'GTV' in filtered_summary.columns:
                    scatter_data = filtered_summary[(filtered_summary['Cost'] > 0) & (filtered_summary['GTV'] > 0)]
                    if not scatter_data.empty:
                        fig2 = px.scatter(scatter_data, 
                                         x='Cost', y='GTV', 
                                         size='Unique Click-Through Conversions',
                                         hover_name='Journey Name',
                                         title="Cost vs GTV Analysis (Filtered)",
                                         labels={'Cost': 'Total Cost (₹)', 'GTV': 'Gross Transaction Value (₹)'})
                    else:
                        fig2 = px.scatter(title="No cost/GTV data available")
                else:
                    fig2 = px.scatter(title="No cost/GTV data available")
                fig2.update_layout(height=400)
                
                # Chart 3: Volume Distribution
                volume_data = filtered_summary.nlargest(8, 'Sent')
                fig3 = px.pie(volume_data, 
                             values='Sent', names='Journey Name',
                             title=f"Top {len(volume_data)} Journeys by Messages Sent")
                fig3.update_layout(height=400)
                
                # Chart 4: Channel Performance for filtered data
                filtered_channel_df = analyzer.calculate_channel_breakdown(filtered_data)
                
                if not filtered_channel_df.empty:
                    channel_totals = {}
                    for channel in ['Email', 'Push', 'SMS', 'WhatsApp']:
                        total = 0
                        try:
                            for _, row in filtered_channel_df.iterrows():
                                val = row.get(f'{channel}_Sent', '0')
                                if val and val != '' and val != '0':
                                    total += int(str(val).replace(',', ''))
                        except:
                            pass
                        channel_totals[channel] = total
                    
                    fig4 = px.bar(x=list(channel_totals.keys()), 
                                 y=list(channel_totals.values()),
                                 title="Messages Sent by Channel (Filtered)",
                                 labels={'x': 'Channel', 'y': 'Total Messages Sent'})
                    fig4.update_layout(height=400)
                else:
                    fig4 = px.bar(title="No channel data available")
                
                return [
                    html.H5(f"📈 Charts for {len(filtered_summary)} filtered journeys", className="mb-3"),
                    dbc.Row([
                        dbc.Col([dcc.Graph(figure=fig1)], width=6),
                        dbc.Col([dcc.Graph(figure=fig2)], width=6),
                        dbc.Col([dcc.Graph(figure=fig3)], width=6),
                        dbc.Col([dcc.Graph(figure=fig4)], width=6)
                    ])
                ]
                
            except Exception as e:
                return dbc.Alert(f"Error creating charts: {str(e)}", color="danger")
        
        elif active_tab == "details":
            # Campaign details view
            detail_cols = ['Campaign Name', 'Journey Name', 'Channel', 'Status', 'Sent', 'Delivered', 'Unique Click-Through Conversions']
            available_detail_cols = [col for col in detail_cols if col in filtered_data.columns]
            
            campaign_details = filtered_data[available_detail_cols].copy()
            
            # Format numeric columns safely
            for col in ['Sent', 'Delivered', 'Unique Click-Through Conversions']:
                if col in campaign_details.columns:
                    campaign_details[col] = campaign_details[col].apply(lambda x: f"{int(x):,}" if pd.notna(x) else "0")
            
            return [
                html.H5(f"📋 Campaign Details ({len(campaign_details)} campaigns)", className="mb-3"),
                dash_table.DataTable(
                    data=campaign_details.to_dict('records'),
                    columns=[{"name": i, "id": i} for i in campaign_details.columns],
                    style_table={'overflowX': 'auto'},
                    style_cell={
                        'textAlign': 'left',
                        'padding': '8px',
                        'fontFamily': 'Arial',
                        'fontSize': '11px'
                    },
                    style_header={
                        'backgroundColor': 'rgb(230, 230, 230)',
                        'fontWeight': 'bold'
                    },
                    sort_action="native",
                    filter_action="native",
                    page_size=25,
                    export_format="xlsx"
                )
            ]
            
    except Exception as e:
        print(f"Error in render_tab_content: {e}")
        return dbc.Alert(f"Error rendering content: {str(e)}", color="danger")

if __name__ == "__main__":
    local_ip = get_local_ip()
    
    print("🚀 Starting Enhanced Campaign Analytics Dashboard...")
    print("📊 Dashboard URLs:")
    print(f"   - Local Access: http://127.0.0.1:8054")
    print(f"   - Network Access: http://{local_ip}:8054")
    print(f"   - WiFi Share URL: http://{local_ip}:8054")
    print("\n📋 Enhanced Features available:")
    print("   ✅ Interactive Journey Selection with checkboxes")
    print("   ✅ Channel-wise filtering")
    print("   ✅ Date range filtering")
    print("   ✅ Performance-based filters")
    print("   ✅ Quick filter buttons")
    print("   ✅ Dynamic metrics calculation")
    print("   ✅ Campaign details view")
    print("   ✅ Real-time table updates")
    print(f"\n🌐 Share this URL with others on your WiFi:")
    print(f"   http://{local_ip}:8054")
    print("\n" + "="*60)
    
    try:
        app.run(
            debug=True,
            host='0.0.0.0',
            port=8054,
            dev_tools_hot_reload=True
        )
    except Exception as e:
        print(f"❌ Error starting server on 0.0.0.0: {e}")
        print("📝 Trying localhost configuration...")
        try:
            app.run(
                debug=True,
                host='127.0.0.1',
                port=8054
            )
        except Exception as e2:
            print(f"❌ Localhost failed: {e2}")
            print("🔧 Try changing the port number and run again")

✅ Data loaded successfully: 28750 records
📊 Summary table: 21 journeys
📋 Channel breakdown: 21 journeys
🚀 Starting Enhanced Campaign Analytics Dashboard...
📊 Dashboard URLs:
   - Local Access: http://127.0.0.1:8054
   - Network Access: http://172.16.56.59:8054
   - WiFi Share URL: http://172.16.56.59:8054

📋 Enhanced Features available:
   ✅ Interactive Journey Selection with checkboxes
   ✅ Channel-wise filtering
   ✅ Date range filtering
   ✅ Performance-based filters
   ✅ Quick filter buttons
   ✅ Dynamic metrics calculation
   ✅ Campaign details view
   ✅ Real-time table updates

🌐 Share this URL with others on your WiFi:
   http://172.16.56.59:8054

❌ Error starting server on 0.0.0.0: HTTPConnectionPool(host='0.0.0.0', port=8054): Max retries exceeded with url: /_alive_ff040fa0-6e53-41b0-837c-9f9e78494978 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001FCF61C6900>: Failed to establish a new connection: [WinError 10049] The requested address is n

In [2]:
import pandas as pd
import numpy as np
import warnings
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import dash
from dash import dcc, html, dash_table, callback
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
import socket

warnings.filterwarnings('ignore')

# Configuration Variables
AOV = 1500  
CHANNEL_COSTS = {
    'Email': 0.02,
    'Push': 0.01, 
    'SMS': 0.11,
    'WhatsApp': 0.11
}

class CampaignAnalyzer:
    def __init__(self, file_path, aov=AOV, channel_costs=CHANNEL_COSTS):
        self.file_path = file_path
        self.aov = aov
        self.channel_costs = channel_costs
        self.raw_data = None
        self.summary_data = None
        self.channel_data = None
        
    def load_and_process_data(self):
        """Load and process campaign data"""
        try:
            df = pd.read_csv(self.file_path)
            df = df.fillna(0)
            
            numeric_columns = ['Sent', 'Delivered', 'Unique Impressions', 'Unique Clicks', 
                              'Unique Conversions', 'Unique Click-Through Conversions']
            
            for col in numeric_columns:
                if col in df.columns:
                    df[col] = pd.to_numeric(
                        df[col].astype(str).str.replace(',', '').str.replace('nan', '0'), 
                        errors='coerce'
                    ).fillna(0)
            
            if 'Day' in df.columns:
                df['Day'] = pd.to_datetime(df['Day'], errors='coerce')
            
            if 'Journey Name' in df.columns:
                df['Journey Name'] = df['Journey Name'].astype(str)
            
            if 'Channel' in df.columns:
                df['Channel'] = df['Channel'].astype(str)
            
            if 'Status' in df.columns:
                df['Status'] = df['Status'].astype(str)
                
            self.raw_data = df
            return df
        except Exception as e:
            print(f"Error loading data: {e}")
            return None
    
    def calculate_summary_metrics(self, filtered_df=None):
        """Calculate main summary table metrics"""
        df = filtered_df if filtered_df is not None else self.raw_data
        
        if df is None or df.empty:
            return pd.DataFrame()
            
        try:
            grouped = df.groupby('Journey Name').agg({
                'Campaign ID': 'count',
                'Sent': 'sum',
                'Delivered': 'sum', 
                'Unique Impressions': 'sum',
                'Unique Clicks': 'sum',
                'Unique Conversions': 'sum',
                'Unique Click-Through Conversions': 'sum'
            }).reset_index()
            
            grouped.rename(columns={'Campaign ID': 'Count of Campaign ID'}, inplace=True)
            
            grouped['v_dr'] = np.where(grouped['Sent'] > 0, 
                                      (grouped['Delivered'] / grouped['Sent'] * 100).round(0), 0)
            
            grouped['v_ctr'] = np.where(grouped['Delivered'] > 0,
                                       (grouped['Unique Clicks'] / grouped['Delivered'] * 100).round(1), 0)
            
            grouped['v_cr'] = np.where(grouped['Unique Clicks'] > 0,
                                      (grouped['Unique Click-Through Conversions'] / grouped['Unique Clicks'] * 100).round(1), 0)
            
            grouped['v_ord_per_sent'] = np.where(grouped['Sent'] > 0,
                                                (grouped['Unique Click-Through Conversions'] / grouped['Sent'] * 100).round(2), 0)
            
            financial_metrics = self.calculate_financial_metrics(filtered_df)
            
            if not financial_metrics.empty:
                summary_df = grouped.merge(financial_metrics[['Journey Name', 'Cost', 'GTV', 'ROI']], 
                                         on='Journey Name', how='left')
            else:
                summary_df = grouped.copy()
                summary_df['Cost'] = 0
                summary_df['GTV'] = 0
                summary_df['ROI'] = 0
            
            summary_df['Row Labels'] = summary_df['Journey Name']
            summary_df['Sum of Sent'] = summary_df['Sent'].apply(lambda x: f"{int(x):,}")
            summary_df['Sum of Delivered'] = summary_df['Delivered'].apply(lambda x: f"{int(x):,}")
            summary_df['Sum of Unique Impressions'] = summary_df['Unique Impressions'].apply(lambda x: f"{int(x):,}")
            summary_df['Sum of Unique Clicks'] = summary_df['Unique Clicks'].apply(lambda x: f"{int(x):,}")
            summary_df['Sum of Unique Click-Through Conversions'] = summary_df['Unique Click-Through Conversions'].apply(lambda x: f"{int(x):,}")
            summary_df['v_dr_formatted'] = summary_df['v_dr'].astype(int).astype(str) + '%'
            summary_df['v_ctr_formatted'] = summary_df['v_ctr'].astype(str) + '%'
            summary_df['v_cr_formatted'] = summary_df['v_cr'].astype(str) + '%'
            summary_df['v_ord_per_sent_formatted'] = summary_df['v_ord_per_sent'].astype(str) + '%'
            summary_df['Cost_formatted'] = summary_df['Cost'].apply(lambda x: f"₹ {x:,.2f}")
            summary_df['GTV_formatted'] = summary_df['GTV'].apply(lambda x: f"{int(x):,}")
            summary_df['ROI_formatted'] = summary_df['ROI'].apply(lambda x: f"₹ {x:.2f}")
            
            return summary_df
            
        except Exception as e:
            print(f"Error in calculate_summary_metrics: {e}")
            return pd.DataFrame()
    
    def calculate_financial_metrics(self, filtered_df=None):
        """Calculate cost, GTV, and ROI metrics"""
        df = filtered_df if filtered_df is not None else self.raw_data
        
        if df is None or df.empty:
            return pd.DataFrame()
        
        try:
            financial_df = df.groupby('Journey Name').agg({
                'Sent': 'sum',
                'Channel': lambda x: list(x),
                'Unique Click-Through Conversions': 'sum'
            }).reset_index()
            
            financial_df['Cost'] = 0
            for idx, row in financial_df.iterrows():
                try:
                    channel_sent = df[df['Journey Name'] == row['Journey Name']].groupby('Channel')['Sent'].sum()
                    total_cost = 0
                    for channel, sent_count in channel_sent.items():
                        if channel in self.channel_costs:
                            total_cost += sent_count * self.channel_costs[channel]
                    financial_df.at[idx, 'Cost'] = total_cost
                except Exception as e:
                    print(f"Error calculating cost for {row['Journey Name']}: {e}")
                    financial_df.at[idx, 'Cost'] = 0
            
            financial_df['GTV'] = financial_df['Unique Click-Through Conversions'] * self.aov
            financial_df['ROI'] = np.where(financial_df['Cost'] > 0,
                                          (financial_df['GTV'] / financial_df['Cost']).round(2), 0)
            
            return financial_df
            
        except Exception as e:
            print(f"Error in calculate_financial_metrics: {e}")
            return pd.DataFrame()
    
    def calculate_channel_breakdown(self, filtered_df=None):
        """Calculate channel-wise breakdown"""
        df = filtered_df if filtered_df is not None else self.raw_data
        
        if df is None or df.empty:
            return pd.DataFrame()
        
        try:
            journeys = df['Journey Name'].unique()
            channels = ['Email', 'Push', 'SMS', 'WhatsApp']
            breakdown_data = []
            
            for journey in journeys:
                journey_data = df[df['Journey Name'] == journey]
                row_data = {'Row Labels': str(journey)}
                
                total_sent = journey_data['Sent'].sum()
                total_delivered = journey_data['Delivered'].sum()
                total_conversions = journey_data['Unique Click-Through Conversions'].sum()
                
                for channel in channels:
                    channel_data = journey_data[journey_data['Channel'] == channel]
                    sent = channel_data['Sent'].sum()
                    delivered = channel_data['Delivered'].sum()
                    conversions = channel_data['Unique Click-Through Conversions'].sum()
                    
                    row_data[f'{channel}_Sent'] = f"{int(sent):,}" if sent > 0 else ""
                    row_data[f'{channel}_Delivered'] = f"{int(delivered):,}" if delivered > 0 else ""
                    row_data[f'{channel}_Conversions'] = f"{int(conversions):,}" if conversions > 0 else ""
                
                row_data['Total_Sent'] = f"{int(total_sent):,}"
                row_data['Total_Delivered'] = f"{int(total_delivered):,}"
                row_data['Total_Conversions'] = f"{int(total_conversions):,}"
                
                breakdown_data.append(row_data)
            
            channel_breakdown = pd.DataFrame(breakdown_data)
            return channel_breakdown
            
        except Exception as e:
            print(f"Error in calculate_channel_breakdown: {e}")
            return pd.DataFrame()

# Initialize the analyzer
try:
    analyzer = CampaignAnalyzer("report-1750749443470_jBRS6FP_Snapmint _in~~c2ab3517.csv")
    data_loaded = analyzer.load_and_process_data()
    
    if data_loaded is not None:
        summary_df = analyzer.calculate_summary_metrics()
        channel_df = analyzer.calculate_channel_breakdown()
        
        print(f"Data loaded successfully: {len(data_loaded)} records")
        print(f"Summary table: {len(summary_df)} journeys")
        print(f"Channel breakdown: {len(channel_df)} journeys")
        
        try:
            unique_journeys = sorted([str(j) for j in data_loaded['Journey Name'].unique() if pd.notna(j)])
            unique_channels = sorted([str(c) for c in data_loaded['Channel'].unique() if pd.notna(c)])
            unique_statuses = sorted([str(s) for s in data_loaded['Status'].unique() if pd.notna(s)]) if 'Status' in data_loaded.columns else []
            
            if 'Day' in data_loaded.columns:
                valid_dates = data_loaded['Day'].dropna()
                if not valid_dates.empty:
                    min_date = valid_dates.min()
                    max_date = valid_dates.max()
                else:
                    min_date = max_date = None
            else:
                min_date = max_date = None
                
        except Exception as e:
            print(f"Error processing filter options: {e}")
            unique_journeys = []
            unique_channels = []
            unique_statuses = []
            min_date = max_date = None
            
    else:
        print("Failed to load data")
        summary_df = pd.DataFrame()
        channel_df = pd.DataFrame()
        unique_journeys = []
        unique_channels = []
        unique_statuses = []
        min_date = max_date = None
        
except Exception as e:
    print(f"Error initializing analyzer: {e}")
    summary_df = pd.DataFrame()
    channel_df = pd.DataFrame()
    unique_journeys = []
    unique_channels = []
    unique_statuses = []
    min_date = max_date = None

# Initialize Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

def get_local_ip():
    """Get the local IP address of the machine"""
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        s.connect(("8.8.8.8", 80))
        local_ip = s.getsockname()[0]
        s.close()
        return local_ip
    except Exception:
        return "127.0.0.1"

# Define the layout
def create_layout():
    if summary_df.empty:
        return dbc.Container([
            html.H1("Campaign Performance Analytics Dashboard", className="text-center mb-4 text-danger"),
            html.Hr(),
            dbc.Alert("Failed to load data. Please check your file path and data format.", color="danger")
        ], fluid=True)
    
    return dbc.Container([
        dbc.Row([
            dbc.Col([
                html.H1("Campaign Performance Analytics Dashboard", 
                       className="text-center mb-4 text-primary"),
                html.Hr()
            ])
        ]),
        
        # Section Visibility Controls
        dbc.Card([
            dbc.CardHeader([
                html.H5("Section Visibility Controls", className="mb-0")
            ]),
            dbc.CardBody([
                dbc.Row([
                    dbc.Col([
                        dbc.Checklist(
                            options=[
                                {"label": "Show Filters", "value": "filters"},
                                {"label": "Show Metrics Cards", "value": "metrics"},
                                {"label": "Show Summary Table", "value": "summary"},
                                {"label": "Show Channel Breakdown", "value": "channels"},
                                {"label": "Show Performance Charts", "value": "charts"},
                                {"label": "Show Campaign Details", "value": "details"},
                            ],
                            value=["filters", "metrics", "summary", "channels", "charts"],
                            id="section-visibility",
                            inline=True,
                            className="mb-2"
                        )
                    ])
                ])
            ])
        ], className="mb-4"),
        
        # Filter Controls Section
        html.Div([
            dbc.Card([
                dbc.CardHeader([
                    html.H5("Filter Controls", className="mb-0")
                ]),
                dbc.CardBody([
                    dbc.Row([
                        # Journey Selection
                        dbc.Col([
                            html.Label("Select Journeys:", className="fw-bold"),
                            html.Div([
                                dbc.Button("Select All", id="select-all-journeys", size="sm", color="outline-primary", className="me-2 mb-2"),
                                dbc.Button("Clear All", id="clear-all-journeys", size="sm", color="outline-secondary", className="me-2 mb-2"),
                                dbc.Button("Top 10 ROI", id="top-roi-journeys", size="sm", color="outline-success", className="mb-2"),
                            ]),
                            dcc.Checklist(
                                id='journey-checklist',
                                options=[{'label': journey, 'value': journey} for journey in unique_journeys],
                                value=unique_journeys[:10] if len(unique_journeys) > 10 else unique_journeys,
                                style={'maxHeight': '200px', 'overflowY': 'scroll', 'border': '1px solid #ccc', 'padding': '10px'},
                                className="mt-2"
                            )
                        ], width=4),
                        
                        # Channel Selection  
                        dbc.Col([
                            html.Label("Select Channels:", className="fw-bold"),
                            html.Div([
                                dbc.Button("Select All", id="select-all-channels", size="sm", color="outline-primary", className="me-2 mb-2"),
                                dbc.Button("Clear All", id="clear-all-channels", size="sm", color="outline-secondary", className="mb-2"),
                            ]),
                            dcc.Checklist(
                                id='channel-checklist',
                                options=[{'label': channel, 'value': channel} for channel in unique_channels],
                                value=unique_channels,
                                className="mt-2"
                            )
                        ], width=2),
                        
                        # Status Selection
                        dbc.Col([
                            html.Label("Campaign Status:", className="fw-bold"),
                            dcc.Checklist(
                                id='status-checklist',
                                options=[{'label': status, 'value': status} for status in unique_statuses],
                                value=unique_statuses,
                                className="mt-2"
                            )
                        ], width=2),
                        
                        # Date Range and Performance Filters
                        dbc.Col([
                            html.Label("Date Range:", className="fw-bold"),
                            dcc.DatePickerRange(
                                id='date-picker-range',
                                start_date=min_date,
                                end_date=max_date,
                                display_format='YYYY-MM-DD',
                                className="mb-3"
                            ) if min_date and max_date else html.P("No date data available"),
                            
                            html.Label("Performance Filter:", className="fw-bold"),
                            dcc.Dropdown(
                                id='performance-filter',
                                options=[
                                    {'label': 'All Campaigns', 'value': 'all'},
                                    {'label': 'High ROI (>10x)', 'value': 'high_roi'},
                                    {'label': 'High CTR (>2%)', 'value': 'high_ctr'},
                                    {'label': 'High Conversion (>1%)', 'value': 'high_conv'},
                                    {'label': 'Low Performers', 'value': 'low_perf'}
                                ],
                                value='all',
                                className="mt-2"
                            )
                        ], width=4),
                    ]),
                    
                    html.Hr(),
                    
                    # Quick Filter Buttons
                    dbc.Row([
                        dbc.Col([
                            html.Label("Quick Filters:", className="fw-bold"),
                            html.Div([
                                dbc.Button("Top Performers", id="quick-top-performers", size="sm", color="success", className="me-2 mb-2"),
                                dbc.Button("Email Only", id="quick-email-only", size="sm", color="info", className="me-2 mb-2"),
                                dbc.Button("Push Only", id="quick-push-only", size="sm", color="warning", className="me-2 mb-2"),
                                dbc.Button("High Value", id="quick-high-value", size="sm", color="primary", className="me-2 mb-2"),
                                dbc.Button("Reset All", id="reset-filters", size="sm", color="secondary", className="mb-2"),
                            ])
                        ])
                    ])
                ])
            ], className="mb-4")
        ], id="filters-section"),
        
        # Key Metrics Row (Dynamic based on filters)
        html.Div(id="metrics-cards-section"),
        
        # Summary Table Section
        html.Div(id="summary-section"),
        
        # Channel Breakdown Section
        html.Div(id="channels-section"),
        
        # Charts Section
        html.Div(id="charts-section"),
        
        # Details Section
        html.Div(id="details-section"),
        
    ], fluid=True)

app.layout = create_layout()

# Callback for section visibility
@app.callback(
    [Output("filters-section", "style"),
     Output("metrics-cards-section", "style"),
     Output("summary-section", "style"),
     Output("channels-section", "style"),
     Output("charts-section", "style"),
     Output("details-section", "style")],
    [Input("section-visibility", "value")]
)
def toggle_sections(visible_sections):
    def get_style(section_id):
        return {} if section_id in visible_sections else {"display": "none"}
    
    return (
        get_style("filters"),
        get_style("metrics"),
        get_style("summary"),
        get_style("channels"),
        get_style("charts"),
        get_style("details")
    )

# Callback for filter interactions
@app.callback(
    [Output('journey-checklist', 'value'),
     Output('channel-checklist', 'value')],
    [Input('select-all-journeys', 'n_clicks'),
     Input('clear-all-journeys', 'n_clicks'),
     Input('top-roi-journeys', 'n_clicks'),
     Input('select-all-channels', 'n_clicks'),
     Input('clear-all-channels', 'n_clicks'),
     Input('quick-top-performers', 'n_clicks'),
     Input('quick-email-only', 'n_clicks'),
     Input('quick-push-only', 'n_clicks'),
     Input('quick-high-value', 'n_clicks'),
     Input('reset-filters', 'n_clicks')],
    [State('journey-checklist', 'value'),
     State('channel-checklist', 'value')],
    prevent_initial_call=True
)
def update_filters(select_all_j, clear_all_j, top_roi_j, select_all_c, clear_all_c,
                  quick_top, quick_email, quick_push, quick_value, reset_all,
                  current_journeys, current_channels):
    
    try:
        ctx = dash.callback_context
        if not ctx.triggered:
            return current_journeys or [], current_channels or []
        
        button_id = ctx.triggered[0]['prop_id'].split('.')[0]
        
        if button_id == 'select-all-journeys':
            return unique_journeys, current_channels or []
        elif button_id == 'clear-all-journeys':
            return [], current_channels or []
        elif button_id == 'top-roi-journeys':
            if not summary_df.empty and 'ROI' in summary_df.columns:
                try:
                    top_journeys = summary_df.nlargest(10, 'ROI')['Journey Name'].tolist()
                    return top_journeys, current_channels or []
                except:
                    pass
            return current_journeys or [], current_channels or []
        elif button_id == 'select-all-channels':
            return current_journeys or [], unique_channels
        elif button_id == 'clear-all-channels':
            return current_journeys or [], []
        elif button_id == 'quick-top-performers':
            if not summary_df.empty and 'ROI' in summary_df.columns:
                try:
                    top_journeys = summary_df.nlargest(5, 'ROI')['Journey Name'].tolist()
                    return top_journeys, current_channels or []
                except:
                    pass
            return current_journeys or [], current_channels or []
        elif button_id == 'quick-email-only':
            return current_journeys or [], ['Email']
        elif button_id == 'quick-push-only':
            return current_journeys or [], ['Push']
        elif button_id == 'quick-high-value':
            if not summary_df.empty and 'GTV' in summary_df.columns:
                try:
                    median_gtv = summary_df['GTV'].median()
                    high_value = summary_df[summary_df['GTV'] > median_gtv]['Journey Name'].tolist()
                    return high_value, current_channels or []
                except:
                    pass
            return current_journeys or [], current_channels or []
        elif button_id == 'reset-filters':
            return unique_journeys, unique_channels
        
        return current_journeys or [], current_channels or []
        
    except Exception as e:
        print(f"Error in update_filters: {e}")
        return current_journeys or [], current_channels or []

# Dynamic metrics cards callback
@app.callback(
    Output("metrics-cards-section", "children"),
    [Input('journey-checklist', 'value'),
     Input('channel-checklist', 'value'),
     Input('status-checklist', 'value'),
     Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date'),
     Input('performance-filter', 'value')]
)
def update_metrics_cards(selected_journeys, selected_channels, selected_statuses, start_date, end_date, perf_filter):
    try:
        if data_loaded.empty:
            return dbc.Alert("No data available", color="warning", className="mb-4")
        
        filtered_data = data_loaded.copy()
        
        if selected_journeys:
            filtered_data = filtered_data[filtered_data['Journey Name'].isin(selected_journeys)]
        if selected_channels:
            filtered_data = filtered_data[filtered_data['Channel'].isin(selected_channels)]
        if selected_statuses:
            filtered_data = filtered_data[filtered_data['Status'].isin(selected_statuses)]
        if start_date and end_date and 'Day' in filtered_data.columns:
            filtered_data = filtered_data[(filtered_data['Day'] >= start_date) & (filtered_data['Day'] <= end_date)]
        
        filtered_summary = analyzer.calculate_summary_metrics(filtered_data)
        
        if filtered_summary.empty:
            return dbc.Alert("No data matches the selected filters.", color="warning", className="mb-4")
        
        total_sent = filtered_summary['Sent'].sum() if 'Sent' in filtered_summary.columns else 0
        total_delivered = filtered_summary['Delivered'].sum() if 'Delivered' in filtered_summary.columns else 0
        total_conversions = filtered_summary['Unique Click-Through Conversions'].sum() if 'Unique Click-Through Conversions' in filtered_summary.columns else 0
        total_cost = filtered_summary['Cost'].sum() if 'Cost' in filtered_summary.columns else 0
        total_gtv = filtered_summary['GTV'].sum() if 'GTV' in filtered_summary.columns else 0
        overall_roi = total_gtv / total_cost if total_cost > 0 else 0
        
        return dbc.Row([
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H4(f"{total_sent:,}", className="text-primary"),
                        html.P("Total Messages Sent", className="mb-0"),
                        html.Small(f"{len(filtered_summary)} journeys", className="text-muted")
                    ])
                ])
            ], width=2),
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H4(f"{total_delivered:,}", className="text-success"),
                        html.P("Total Delivered", className="mb-0"),
                        html.Small(f"{(total_delivered/total_sent*100 if total_sent > 0 else 0):.1f}% delivery rate", className="text-muted")
                    ])
                ])
            ], width=2),
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H4(f"{total_conversions:,}", className="text-info"),
                        html.P("Total Conversions", className="mb-0"),
                        html.Small(f"{(total_conversions/total_delivered*100 if total_delivered > 0 else 0):.2f}% conv rate", className="text-muted")
                    ])
                ])
            ], width=2),
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H4(f"₹{total_cost:,.0f}", className="text-warning"),
                        html.P("Total Cost", className="mb-0"),
                        html.Small(f"₹{(total_cost/total_sent if total_sent > 0 else 0):.3f} per message", className="text-muted")
                    ])
                ])
            ], width=2),
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H4(f"₹{total_gtv:,.0f}", className="text-success"),
                        html.P("Total GTV", className="mb-0"),
                        html.Small(f"₹{(total_gtv/total_conversions if total_conversions > 0 else 0):,.0f} per conversion", className="text-muted")
                    ])
                ])
            ], width=2),
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H4(f"{overall_roi:.2f}x", className="text-danger"),
                        html.P("Overall ROI", className="mb-0"),
                        html.Small(f"{'Profitable' if overall_roi > 1 else 'Loss'}", className="text-muted")
                    ])
                ])
            ], width=2),
        ], className="mb-4")
        
    except Exception as e:
        print(f"Error in update_metrics_cards: {e}")
        return dbc.Alert("Error calculating metrics", color="warning", className="mb-4")

# Summary table callback
@app.callback(
    Output("summary-section", "children"),
    [Input('journey-checklist', 'value'),
     Input('channel-checklist', 'value'),
     Input('status-checklist', 'value'),
     Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date'),
     Input('performance-filter', 'value')]
)
def update_summary_table(selected_journeys, selected_channels, selected_statuses, start_date, end_date, perf_filter):
    try:
        if data_loaded.empty:
            return dbc.Alert("No data available to display.", color="warning")
        
        filtered_data = data_loaded.copy()
        
        if selected_journeys:
            filtered_data = filtered_data[filtered_data['Journey Name'].isin(selected_journeys)]
        if selected_channels:
            filtered_data = filtered_data[filtered_data['Channel'].isin(selected_channels)]
        if selected_statuses:
            filtered_data = filtered_data[filtered_data['Status'].isin(selected_statuses)]
        if start_date and end_date and 'Day' in filtered_data.columns:
            filtered_data = filtered_data[(filtered_data['Day'] >= start_date) & (filtered_data['Day'] <= end_date)]
        
        filtered_summary = analyzer.calculate_summary_metrics(filtered_data)
        
        if perf_filter == 'high_roi' and 'ROI' in filtered_summary.columns:
            filtered_summary = filtered_summary[filtered_summary['ROI'] > 10]
        elif perf_filter == 'high_ctr' and 'v_ctr' in filtered_summary.columns:
            filtered_summary = filtered_summary[filtered_summary['v_ctr'] > 2]
        elif perf_filter == 'high_conv' and 'v_ord_per_sent' in filtered_summary.columns:
            filtered_summary = filtered_summary[filtered_summary['v_ord_per_sent'] > 1]
        elif perf_filter == 'low_perf' and 'ROI' in filtered_summary.columns:
            filtered_summary = filtered_summary[filtered_summary['ROI'] < 5]
        
        if filtered_summary.empty:
            return dbc.Alert("No data matches the selected filters.", color="warning")
        
        available_cols = [
            'Row Labels', 'Count of Campaign ID', 'Sum of Sent', 'Sum of Delivered',
            'Sum of Unique Impressions', 'Sum of Unique Clicks', 
            'Sum of Unique Click-Through Conversions', 'v_dr_formatted', 
            'v_ctr_formatted', 'v_cr_formatted', 'v_ord_per_sent_formatted',
            'Cost_formatted', 'GTV_formatted', 'ROI_formatted'
        ]
        
        display_cols = [col for col in available_cols if col in filtered_summary.columns]
        summary_display_df = filtered_summary[display_cols].copy()
        
        column_names = {
            'Row Labels': 'Row Labels',
            'Count of Campaign ID': 'Count of Campaign ID',
            'Sum of Sent': 'Sum of Sent',
            'Sum of Delivered': 'Sum of Delivered',
            'Sum of Unique Impressions': 'Sum of Unique Impressions',
            'Sum of Unique Clicks': 'Sum of Unique Clicks',
            'Sum of Unique Click-Through Conversions': 'Sum of Unique Click-Through Conversions',
            'v_dr_formatted': 'v_dr',
            'v_ctr_formatted': 'v_ctr',
            'v_cr_formatted': 'v_cr',
            'v_ord_per_sent_formatted': 'v_ord_per_sent',
            'Cost_formatted': 'Cost',
            'GTV_formatted': 'GTV',
            'ROI_formatted': 'ROI'
        }
        
        summary_display_df.columns = [column_names.get(col, col) for col in summary_display_df.columns]
        
        return dbc.Card([
            dbc.CardHeader([
                html.H5(f"Summary Table - Showing {len(summary_display_df)} journeys (filtered from {len(summary_df)} total)", className="mb-0")
            ]),
            dbc.CardBody([
                dash_table.DataTable(
                    data=summary_display_df.to_dict('records'),
                    columns=[{"name": i, "id": i} for i in summary_display_df.columns],
                    style_table={'overflowX': 'auto'},
                    style_cell={
                        'textAlign': 'left',
                        'padding': '10px',
                        'fontFamily': 'Arial',
                        'fontSize': '12px'
                    },
                    style_header={
                        'backgroundColor': 'rgb(230, 230, 230)',
                        'fontWeight': 'bold',
                        'fontSize': '14px'
                    },
                    style_data_conditional=[
                        {
                            'if': {'row_index': 'odd'},
                            'backgroundColor': 'rgb(248, 248, 248)'
                        }
                    ],
                    sort_action="native",
                    filter_action="native",
                    page_action="none",  # Remove pagination - show all data
                    export_format="xlsx",
                    export_headers="display"
                )
            ])
        ], className="mb-4")
        
    except Exception as e:
        print(f"Error in update_summary_table: {e}")
        return dbc.Alert(f"Error rendering summary table: {str(e)}", color="danger")

# Channel breakdown callback
@app.callback(
    Output("channels-section", "children"),
    [Input('journey-checklist', 'value'),
     Input('channel-checklist', 'value'),
     Input('status-checklist', 'value'),
     Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date')]
)
def update_channel_breakdown(selected_journeys, selected_channels, selected_statuses, start_date, end_date):
    try:
        if data_loaded.empty:
            return dbc.Alert("No data available to display.", color="warning")
        
        filtered_data = data_loaded.copy()
        
        if selected_journeys:
            filtered_data = filtered_data[filtered_data['Journey Name'].isin(selected_journeys)]
        if selected_channels:
            filtered_data = filtered_data[filtered_data['Channel'].isin(selected_channels)]
        if selected_statuses:
            filtered_data = filtered_data[filtered_data['Status'].isin(selected_statuses)]
        if start_date and end_date and 'Day' in filtered_data.columns:
            filtered_data = filtered_data[(filtered_data['Day'] >= start_date) & (filtered_data['Day'] <= end_date)]
        
        filtered_channel_df = analyzer.calculate_channel_breakdown(filtered_data)
        
        if filtered_channel_df.empty:
            return dbc.Alert("No channel breakdown data available for selected filters.", color="warning")
        
        return dbc.Card([
            dbc.CardHeader([
                html.H5(f"Channel Breakdown - {len(filtered_channel_df)} journeys", className="mb-0")
            ]),
            dbc.CardBody([
                dash_table.DataTable(
                    data=filtered_channel_df.to_dict('records'),
                    columns=[{"name": i, "id": i} for i in filtered_channel_df.columns],
                    style_table={'overflowX': 'auto'},
                    style_cell={
                        'textAlign': 'left',
                        'padding': '8px',
                        'fontFamily': 'Arial',
                        'fontSize': '11px',
                        'minWidth': '80px'
                    },
                    style_header={
                        'backgroundColor': 'rgb(230, 230, 230)',
                        'fontWeight': 'bold',
                        'fontSize': '12px'
                    },
                    style_data_conditional=[
                        {
                            'if': {'row_index': 'odd'},
                            'backgroundColor': 'rgb(248, 248, 248)'
                        }
                    ],
                    sort_action="native",
                    filter_action="native",
                    page_action="none",  # Remove pagination - show all data
                    export_format="xlsx",
                    export_headers="display"
                )
            ])
        ], className="mb-4")
        
    except Exception as e:
        print(f"Error in update_channel_breakdown: {e}")
        return dbc.Alert(f"Error rendering channel breakdown: {str(e)}", color="danger")

# Charts callback - Vertical layout
@app.callback(
    Output("charts-section", "children"),
    [Input('journey-checklist', 'value'),
     Input('channel-checklist', 'value'),
     Input('status-checklist', 'value'),
     Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date'),
     Input('performance-filter', 'value')]
)
def update_charts(selected_journeys, selected_channels, selected_statuses, start_date, end_date, perf_filter):
    try:
        if data_loaded.empty:
            return dbc.Alert("No data available to display.", color="warning")
        
        filtered_data = data_loaded.copy()
        
        if selected_journeys:
            filtered_data = filtered_data[filtered_data['Journey Name'].isin(selected_journeys)]
        if selected_channels:
            filtered_data = filtered_data[filtered_data['Channel'].isin(selected_channels)]
        if selected_statuses:
            filtered_data = filtered_data[filtered_data['Status'].isin(selected_statuses)]
        if start_date and end_date and 'Day' in filtered_data.columns:
            filtered_data = filtered_data[(filtered_data['Day'] >= start_date) & (filtered_data['Day'] <= end_date)]
        
        filtered_summary = analyzer.calculate_summary_metrics(filtered_data)
        
        if filtered_summary.empty:
            return dbc.Alert("No data available for charts with current filters.", color="warning")
        
        # Chart 1: Top ROI Performance
        if 'ROI' in filtered_summary.columns:
            chart_data = filtered_summary[filtered_summary['ROI'] > 0].nlargest(15, 'ROI')
            fig1 = px.bar(chart_data, 
                         x='Journey Name', y='ROI', 
                         title=f"Top {len(chart_data)} Journeys by ROI (Filtered)",
                         labels={'ROI': 'Return on Investment'})
        else:
            chart_data = filtered_summary.head(15)
            fig1 = px.bar(chart_data, 
                         x='Journey Name', y='Sent', 
                         title=f"Top {len(chart_data)} Journeys by Messages Sent (Filtered)")
        fig1.update_layout(xaxis_tickangle=45, height=600, showlegend=False)
        
        # Chart 2: Cost vs GTV Analysis
        if 'Cost' in filtered_summary.columns and 'GTV' in filtered_summary.columns:
            scatter_data = filtered_summary[(filtered_summary['Cost'] > 0) & (filtered_summary['GTV'] > 0)]
            if not scatter_data.empty:
                fig2 = px.scatter(scatter_data, 
                                 x='Cost', y='GTV', 
                                 size='Unique Click-Through Conversions',
                                 hover_name='Journey Name',
                                 title="Cost vs GTV Analysis (Filtered)",
                                 labels={'Cost': 'Total Cost (₹)', 'GTV': 'Gross Transaction Value (₹)'})
            else:
                fig2 = px.scatter(title="No cost/GTV data available")
        else:
            fig2 = px.scatter(title="No cost/GTV data available")
        fig2.update_layout(height=600)
        
        # Chart 3: Volume Distribution
        volume_data = filtered_summary.nlargest(10, 'Sent')
        fig3 = px.pie(volume_data, 
                     values='Sent', names='Journey Name',
                     title=f"Top {len(volume_data)} Journeys by Messages Sent")
        fig3.update_layout(height=600)
        
        # Chart 4: Channel Performance
        filtered_channel_df = analyzer.calculate_channel_breakdown(filtered_data)
        
        if not filtered_channel_df.empty:
            channel_totals = {}
            for channel in ['Email', 'Push', 'SMS', 'WhatsApp']:
                total = 0
                try:
                    for _, row in filtered_channel_df.iterrows():
                        val = row.get(f'{channel}_Sent', '0')
                        if val and val != '' and val != '0':
                            total += int(str(val).replace(',', ''))
                except:
                    pass
                channel_totals[channel] = total
            
            fig4 = px.bar(x=list(channel_totals.keys()), 
                         y=list(channel_totals.values()),
                         title="Messages Sent by Channel (Filtered)",
                         labels={'x': 'Channel', 'y': 'Total Messages Sent'})
            fig4.update_layout(height=600, showlegend=False)
        else:
            fig4 = px.bar(title="No channel data available")
        
        return dbc.Card([
            dbc.CardHeader([
                html.H5(f"Performance Charts - {len(filtered_summary)} filtered journeys", className="mb-0")
            ]),
            dbc.CardBody([
                # Vertical layout - one chart below another
                dbc.Row([
                    dbc.Col([dcc.Graph(figure=fig1)], width=12)
                ], className="mb-4"),
                dbc.Row([
                    dbc.Col([dcc.Graph(figure=fig2)], width=12)
                ], className="mb-4"),
                dbc.Row([
                    dbc.Col([dcc.Graph(figure=fig3)], width=12)
                ], className="mb-4"),
                dbc.Row([
                    dbc.Col([dcc.Graph(figure=fig4)], width=12)
                ])
            ])
        ], className="mb-4")
        
    except Exception as e:
        print(f"Error in update_charts: {e}")
        return dbc.Alert(f"Error creating charts: {str(e)}", color="danger")

# Details callback
@app.callback(
    Output("details-section", "children"),
    [Input('journey-checklist', 'value'),
     Input('channel-checklist', 'value'),
     Input('status-checklist', 'value'),
     Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date')]
)
def update_details(selected_journeys, selected_channels, selected_statuses, start_date, end_date):
    try:
        if data_loaded.empty:
            return dbc.Alert("No data available to display.", color="warning")
        
        filtered_data = data_loaded.copy()
        
        if selected_journeys:
            filtered_data = filtered_data[filtered_data['Journey Name'].isin(selected_journeys)]
        if selected_channels:
            filtered_data = filtered_data[filtered_data['Channel'].isin(selected_channels)]
        if selected_statuses:
            filtered_data = filtered_data[filtered_data['Status'].isin(selected_statuses)]
        if start_date and end_date and 'Day' in filtered_data.columns:
            filtered_data = filtered_data[(filtered_data['Day'] >= start_date) & (filtered_data['Day'] <= end_date)]
        
        detail_cols = ['Campaign Name', 'Journey Name', 'Channel', 'Status', 'Sent', 'Delivered', 'Unique Click-Through Conversions']
        available_detail_cols = [col for col in detail_cols if col in filtered_data.columns]
        
        campaign_details = filtered_data[available_detail_cols].copy()
        
        for col in ['Sent', 'Delivered', 'Unique Click-Through Conversions']:
            if col in campaign_details.columns:
                campaign_details[col] = campaign_details[col].apply(lambda x: f"{int(x):,}" if pd.notna(x) else "0")
        
        return dbc.Card([
            dbc.CardHeader([
                html.H5(f"Campaign Details - {len(campaign_details)} campaigns", className="mb-0")
            ]),
            dbc.CardBody([
                dash_table.DataTable(
                    data=campaign_details.to_dict('records'),
                    columns=[{"name": i, "id": i} for i in campaign_details.columns],
                    style_table={'overflowX': 'auto'},
                    style_cell={
                        'textAlign': 'left',
                        'padding': '8px',
                        'fontFamily': 'Arial',
                        'fontSize': '11px'
                    },
                    style_header={
                        'backgroundColor': 'rgb(230, 230, 230)',
                        'fontWeight': 'bold'
                    },
                    sort_action="native",
                    filter_action="native",
                    page_action="none",  # Remove pagination - show all data
                    export_format="xlsx"
                )
            ])
        ], className="mb-4")
        
    except Exception as e:
        print(f"Error in update_details: {e}")
        return dbc.Alert(f"Error rendering details: {str(e)}", color="danger")

if __name__ == "__main__":
    local_ip = get_local_ip()
    
    print("Starting Enhanced Campaign Analytics Dashboard...")
    print("Dashboard URLs:")
    print(f"   - Local Access: http://127.0.0.1:8055")
    print(f"   - Network Access: http://{local_ip}:8055")
    print(f"   - WiFi Share URL: http://{local_ip}:8055")
    print("\nEnhanced Features available:")
    print("   - All data displayed without pagination")
    print("   - Section visibility controls")
    print("   - Interactive filtering")
    print("   - Vertical chart layout")
    print("   - Professional design")
    print(f"\nShare this URL with others on your WiFi:")
    print(f"   http://{local_ip}:8055")
    print("=" * 60)
    
    try:
        app.run(
            debug=True,
            host='0.0.0.0',
            port=8055,
            dev_tools_hot_reload=True
        )
    except Exception as e:
        print(f"Error starting server on 0.0.0.0: {e}")
        print("Trying localhost configuration...")
        try:
            app.run(
                debug=True,
                host='127.0.0.1',
                port=8055
            )
        except Exception as e2:
            print(f"Localhost failed: {e2}")
            print("Try changing the port number and run again")

Data loaded successfully: 28750 records
Summary table: 21 journeys
Channel breakdown: 21 journeys
Starting Enhanced Campaign Analytics Dashboard...
Dashboard URLs:
   - Local Access: http://127.0.0.1:8055
   - Network Access: http://172.16.56.59:8055
   - WiFi Share URL: http://172.16.56.59:8055

Enhanced Features available:
   - All data displayed without pagination
   - Section visibility controls
   - Interactive filtering
   - Vertical chart layout
   - Professional design

Share this URL with others on your WiFi:
   http://172.16.56.59:8055
Error starting server on 0.0.0.0: HTTPConnectionPool(host='0.0.0.0', port=8055): Max retries exceeded with url: /_alive_2064f1d4-3e91-44a3-a50d-5192a20123eb (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000013164F64B90>: Failed to establish a new connection: [WinError 10049] The requested address is not valid in its context'))
Trying localhost configuration...


In [ ]:
import pandas as pd
import numpy as np
import warnings
import plotly.graph_objects as go
import plotly.express as px
import dash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
import socket

warnings.filterwarnings('ignore')

# Configuration Variables
AOV = 1500  
CHANNEL_COSTS = {
    'Email': 0.02,
    'Push': 0.01, 
    'SMS': 0.11,
    'WhatsApp': 0.11
}

class CampaignAnalyzer:
    def __init__(self, file_path, aov=AOV, channel_costs=CHANNEL_COSTS):
        self.file_path = file_path
        self.aov = aov
        self.channel_costs = channel_costs
        self.raw_data = None
        
    def load_and_process_data(self):
        """Load and process campaign data"""
        try:
            df = pd.read_csv(self.file_path)
            df = df.fillna(0)
            
            numeric_columns = ['Sent', 'Delivered', 'Unique Impressions', 'Unique Clicks', 
                              'Unique Conversions', 'Unique Click-Through Conversions']
            
            for col in numeric_columns:
                if col in df.columns:
                    df[col] = pd.to_numeric(
                        df[col].astype(str).str.replace(',', '').str.replace('nan', '0'), 
                        errors='coerce'
                    ).fillna(0)
            
            if 'Journey Name' in df.columns:
                df['Journey Name'] = df['Journey Name'].astype(str)
            
            if 'Channel' in df.columns:
                df['Channel'] = df['Channel'].astype(str)
                
            self.raw_data = df
            return df
        except Exception as e:
            print(f"Error loading data: {e}")
            return None
    
    def calculate_summary_metrics(self, filtered_df=None):
        """Calculate main summary table metrics"""
        df = filtered_df if filtered_df is not None else self.raw_data
        
        if df is None or df.empty:
            return pd.DataFrame()
            
        try:
            grouped = df.groupby('Journey Name').agg({
                'Campaign ID': 'count',
                'Sent': 'sum',
                'Delivered': 'sum', 
                'Unique Impressions': 'sum',
                'Unique Clicks': 'sum',
                'Unique Conversions': 'sum',
                'Unique Click-Through Conversions': 'sum'
            }).reset_index()
            
            grouped.rename(columns={'Campaign ID': 'Count of Campaign ID'}, inplace=True)
            
            # Calculate performance metrics
            grouped['Delivery Rate'] = np.where(grouped['Sent'] > 0, 
                                      (grouped['Delivered'] / grouped['Sent'] * 100).round(1), 0)
            
            grouped['CTR'] = np.where(grouped['Delivered'] > 0,
                                       (grouped['Unique Clicks'] / grouped['Delivered'] * 100).round(2), 0)
            
            grouped['Conversion Rate'] = np.where(grouped['Unique Clicks'] > 0,
                                      (grouped['Unique Click-Through Conversions'] / grouped['Unique Clicks'] * 100).round(2), 0)
            
            # Calculate costs and ROI
            grouped['Cost'] = 0
            grouped['GTV'] = grouped['Unique Click-Through Conversions'] * self.aov
            grouped['ROI'] = 0
            
            for idx, row in grouped.iterrows():
                try:
                    journey_data = df[df['Journey Name'] == row['Journey Name']]
                    total_cost = 0
                    for _, campaign in journey_data.iterrows():
                        channel = campaign['Channel']
                        sent = campaign['Sent']
                        if channel in self.channel_costs:
                            total_cost += sent * self.channel_costs[channel]
                    
                    grouped.at[idx, 'Cost'] = total_cost
                    if total_cost > 0:
                        grouped.at[idx, 'ROI'] = (row['GTV'] / total_cost)
                except:
                    pass
            
            return grouped
            
        except Exception as e:
            print(f"Error in calculate_summary_metrics: {e}")
            return pd.DataFrame()

# Initialize the analyzer
try:
    analyzer = CampaignAnalyzer("report-1750749443470_jBRS6FP_Snapmint _in~~c2ab3517.csv")
    data_loaded = analyzer.load_and_process_data()
    
    if data_loaded is not None:
        summary_df = analyzer.calculate_summary_metrics()
        
        print(f"✅ Data loaded successfully: {len(data_loaded)} records")
        print(f"📊 Summary table: {len(summary_df)} journeys")
        
        unique_journeys = sorted([str(j) for j in data_loaded['Journey Name'].unique() if pd.notna(j)])
        unique_channels = sorted([str(c) for c in data_loaded['Channel'].unique() if pd.notna(c)])
            
    else:
        print("❌ Failed to load data")
        summary_df = pd.DataFrame()
        unique_journeys = []
        unique_channels = []
        
except Exception as e:
    print(f"❌ Error initializing analyzer: {e}")
    summary_df = pd.DataFrame()
    unique_journeys = []
    unique_channels = []

# Initialize Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

def get_local_ip():
    """Get the local IP address of the machine"""
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        s.connect(("8.8.8.8", 80))
        local_ip = s.getsockname()[0]
        s.close()
        return local_ip
    except Exception:
        return "127.0.0.1"

# Define the layout with collapsible sections
app.layout = dbc.Container([
    # Header
    dbc.Row([
        dbc.Col([
            html.H1("📊 Campaign Performance Analytics Dashboard", 
                   className="text-center mb-4 text-primary"),
            html.Hr()
        ])
    ]),
    
    # Always visible KPI Cards
    html.Div(id="kpi-cards", className="mb-4"),
    
    # Collapsible Sections
    html.Div([
        
        # 1. Filter Controls Section
        dbc.Card([
            dbc.CardHeader([
                dbc.Button(
                    "🔍 Filter Controls",
                    id="collapse-filters-btn",
                    color="primary",
                    className="btn-block text-left",
                    style={"border": "none", "background": "none", "color": "inherit", "font-weight": "bold"}
                )
            ]),
            dbc.Collapse([
                dbc.CardBody([
                    dbc.Row([
                        dbc.Col([
                            html.Label("Select Journeys:", className="fw-bold"),
                            dcc.Dropdown(
                                id='journey-dropdown',
                                options=[{'label': journey, 'value': journey} for journey in unique_journeys[:20]],
                                value=unique_journeys[:5] if len(unique_journeys) >= 5 else unique_journeys,
                                multi=True,
                                placeholder="Select journeys..."
                            )
                        ], width=6),
                        dbc.Col([
                            html.Label("Select Channels:", className="fw-bold"),
                            dcc.Dropdown(
                                id='channel-dropdown',
                                options=[{'label': channel, 'value': channel} for channel in unique_channels],
                                value=unique_channels,
                                multi=True,
                                placeholder="Select channels..."
                            )
                        ], width=6)
                    ])
                ])
            ], id="collapse-filters", is_open=False)
        ], className="mb-3"),
        
        # 2. Quick Filters Section
        dbc.Card([
            dbc.CardHeader([
                dbc.Button(
                    "⚡ Quick Filters",
                    id="collapse-quick-btn",
                    color="info",
                    className="btn-block text-left",
                    style={"border": "none", "background": "none", "color": "inherit", "font-weight": "bold"}
                )
            ]),
            dbc.Collapse([
                dbc.CardBody([
                    dbc.Row([
                        dbc.Col([
                            dbc.Button("🏆 Top 5 ROI", id="quick-top-roi", size="sm", color="success", className="me-2 mb-2"),
                            dbc.Button("📧 Email Only", id="quick-email", size="sm", color="info", className="me-2 mb-2"),
                            dbc.Button("📱 Push Only", id="quick-push", size="sm", color="warning", className="me-2 mb-2"),
                            dbc.Button("🔄 Reset All", id="quick-reset", size="sm", color="secondary", className="mb-2"),
                        ])
                    ])
                ])
            ], id="collapse-quick", is_open=False)
        ], className="mb-3"),
        
        # 3. Summary Table Section
        dbc.Card([
            dbc.CardHeader([
                dbc.Button(
                    "📋 Summary Table",
                    id="collapse-table-btn",
                    color="success",
                    className="btn-block text-left",
                    style={"border": "none", "background": "none", "color": "inherit", "font-weight": "bold"}
                )
            ]),
            dbc.Collapse([
                dbc.CardBody([
                    html.Div(id="summary-table")
                ])
            ], id="collapse-table", is_open=True)  # Open by default
        ], className="mb-3"),
        
        # 4. Charts Section
        dbc.Card([
            dbc.CardHeader([
                dbc.Button(
                    "📈 Performance Charts",
                    id="collapse-charts-btn",
                    color="warning",
                    className="btn-block text-left",
                    style={"border": "none", "background": "none", "color": "inherit", "font-weight": "bold"}
                )
            ]),
            dbc.Collapse([
                dbc.CardBody([
                    html.Div(id="charts-content")
                ])
            ], id="collapse-charts", is_open=False)
        ], className="mb-3"),
        
        # 5. Channel Breakdown Section
        dbc.Card([
            dbc.CardHeader([
                dbc.Button(
                    "📡 Channel Breakdown",
                    id="collapse-channels-btn",
                    color="danger",
                    className="btn-block text-left",
                    style={"border": "none", "background": "none", "color": "inherit", "font-weight": "bold"}
                )
            ]),
            dbc.Collapse([
                dbc.CardBody([
                    html.Div(id="channel-breakdown")
                ])
            ], id="collapse-channels", is_open=False)
        ], className="mb-3"),
        
    ])
    
], fluid=True)

# Collapse button callbacks
@app.callback(
    Output("collapse-filters", "is_open"),
    [Input("collapse-filters-btn", "n_clicks")],
    [State("collapse-filters", "is_open")],
)
def toggle_filters_collapse(n, is_open):
    if n:
        return not is_open
    return is_open

@app.callback(
    Output("collapse-quick", "is_open"),
    [Input("collapse-quick-btn", "n_clicks")],
    [State("collapse-quick", "is_open")],
)
def toggle_quick_collapse(n, is_open):
    if n:
        return not is_open
    return is_open

@app.callback(
    Output("collapse-table", "is_open"),
    [Input("collapse-table-btn", "n_clicks")],
    [State("collapse-table", "is_open")],
)
def toggle_table_collapse(n, is_open):
    if n:
        return not is_open
    return is_open

@app.callback(
    Output("collapse-charts", "is_open"),
    [Input("collapse-charts-btn", "n_clicks")],
    [State("collapse-charts", "is_open")],
)
def toggle_charts_collapse(n, is_open):
    if n:
        return not is_open
    return is_open

@app.callback(
    Output("collapse-channels", "is_open"),
    [Input("collapse-channels-btn", "n_clicks")],
    [State("collapse-channels", "is_open")],
)
def toggle_channels_collapse(n, is_open):
    if n:
        return not is_open
    return is_open

# Quick filter callbacks
@app.callback(
    [Output('journey-dropdown', 'value'),
     Output('channel-dropdown', 'value')],
    [Input('quick-top-roi', 'n_clicks'),
     Input('quick-email', 'n_clicks'),
     Input('quick-push', 'n_clicks'),
     Input('quick-reset', 'n_clicks')],
    [State('journey-dropdown', 'value'),
     State('channel-dropdown', 'value')],
    prevent_initial_call=True
)
def update_quick_filters(top_roi_clicks, email_clicks, push_clicks, reset_clicks, 
                        current_journeys, current_channels):
    ctx = dash.callback_context
    if not ctx.triggered:
        return current_journeys, current_channels
    
    button_id = ctx.triggered[0]['prop_id'].split('.')[0]
    
    if button_id == 'quick-top-roi':
        if not summary_df.empty:
            top_journeys = summary_df.nlargest(5, 'ROI')['Journey Name'].tolist()
            return top_journeys, current_channels
    elif button_id == 'quick-email':
        return current_journeys, ['Email']
    elif button_id == 'quick-push':
        return current_journeys, ['Push']
    elif button_id == 'quick-reset':
        return unique_journeys[:5], unique_channels
    
    return current_journeys, current_channels

# KPI Cards callback
@app.callback(
    Output("kpi-cards", "children"),
    [Input('journey-dropdown', 'value'),
     Input('channel-dropdown', 'value')]
)
def update_kpi_cards(selected_journeys, selected_channels):
    try:
        if not selected_journeys or not selected_channels:
            return dbc.Alert("Please select journeys and channels using the Filter Controls section", 
                           color="info", className="text-center")
        
        # Filter data
        filtered_data = data_loaded[
            (data_loaded['Journey Name'].isin(selected_journeys)) & 
            (data_loaded['Channel'].isin(selected_channels))
        ]
        
        if filtered_data.empty:
            return dbc.Alert("No data matches your selection", color="warning")
        
        # Calculate totals
        total_sent = filtered_data['Sent'].sum()
        total_delivered = filtered_data['Delivered'].sum()
        total_clicks = filtered_data['Unique Clicks'].sum()
        total_conversions = filtered_data['Unique Click-Through Conversions'].sum()
        
        # Calculate costs
        total_cost = 0
        for _, row in filtered_data.iterrows():
            if row['Channel'] in CHANNEL_COSTS:
                total_cost += row['Sent'] * CHANNEL_COSTS[row['Channel']]
        
        total_gtv = total_conversions * AOV
        overall_roi = total_gtv / total_cost if total_cost > 0 else 0
        
        return dbc.Row([
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H4(f"{total_sent:,}", className="text-primary text-center"),
                        html.P("Messages Sent", className="text-center mb-0")
                    ])
                ])
            ], width=2),
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H4(f"{total_delivered:,}", className="text-success text-center"),
                        html.P("Delivered", className="text-center mb-0"),
                        html.Small(f"{(total_delivered/total_sent*100 if total_sent > 0 else 0):.1f}%", 
                                 className="text-muted text-center d-block")
                    ])
                ])
            ], width=2),
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H4(f"{total_clicks:,}", className="text-info text-center"),
                        html.P("Clicks", className="text-center mb-0"),
                        html.Small(f"{(total_clicks/total_delivered*100 if total_delivered > 0 else 0):.2f}%", 
                                 className="text-muted text-center d-block")
                    ])
                ])
            ], width=2),
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H4(f"{total_conversions:,}", className="text-warning text-center"),
                        html.P("Conversions", className="text-center mb-0"),
                        html.Small(f"{(total_conversions/total_clicks*100 if total_clicks > 0 else 0):.2f}%", 
                                 className="text-muted text-center d-block")
                    ])
                ])
            ], width=2),
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H4(f"₹{total_cost:,.0f}", className="text-danger text-center"),
                        html.P("Total Cost", className="text-center mb-0")
                    ])
                ])
            ], width=2),
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H4(f"{overall_roi:.1f}x", className="text-dark text-center"),
                        html.P("ROI", className="text-center mb-0"),
                        html.Small(f"₹{total_gtv:,.0f} GTV", 
                                 className="text-muted text-center d-block")
                    ])
                ])
            ], width=2),
        ])
        
    except Exception as e:
        return dbc.Alert(f"Error calculating metrics: {str(e)}", color="danger")

# Summary table callback
@app.callback(
    Output("summary-table", "children"),
    [Input('journey-dropdown', 'value'),
     Input('channel-dropdown', 'value')]
)
def update_summary_table(selected_journeys, selected_channels):
    try:
        if not selected_journeys or not selected_channels:
            return html.Div("Select filters to view summary table", className="text-center text-muted p-4")
        
        # Filter data
        filtered_data = data_loaded[
            (data_loaded['Journey Name'].isin(selected_journeys)) & 
            (data_loaded['Channel'].isin(selected_channels))
        ]
        
        if filtered_data.empty:
            return html.Div("No data matches your selection", className="text-center text-warning p-4")
        
        # Get summary for selected data
        filtered_summary = analyzer.calculate_summary_metrics(filtered_data)
        
        if filtered_summary.empty:
            return html.Div("No summary data available", className="text-center text-warning p-4")
        
        # Format display columns
        display_df = filtered_summary[[
            'Journey Name', 'Count of Campaign ID', 'Sent', 'Delivered', 
            'Unique Clicks', 'Unique Click-Through Conversions', 
            'Delivery Rate', 'CTR', 'Conversion Rate', 'Cost', 'GTV', 'ROI'
        ]].copy()
        
        # Format numbers
        for col in ['Sent', 'Delivered', 'Unique Clicks', 'Unique Click-Through Conversions']:
            display_df[col] = display_df[col].apply(lambda x: f"{int(x):,}")
        
        for col in ['Delivery Rate', 'CTR', 'Conversion Rate']:
            display_df[col] = display_df[col].apply(lambda x: f"{x:.1f}%")
        
        display_df['Cost'] = display_df['Cost'].apply(lambda x: f"₹{x:,.0f}")
        display_df['GTV'] = display_df['GTV'].apply(lambda x: f"₹{x:,.0f}")
        display_df['ROI'] = display_df['ROI'].apply(lambda x: f"{x:.1f}x")
        
        return [
            html.H5(f"📊 Summary for {len(display_df)} selected journeys", className="mb-3"),
            dash_table.DataTable(
                data=display_df.to_dict('records'),
                columns=[{"name": col, "id": col} for col in display_df.columns],
                style_table={'overflowX': 'auto'},
                style_cell={'textAlign': 'left', 'padding': '10px'},
                style_header={'backgroundColor': 'rgb(230, 230, 230)', 'fontWeight': 'bold'},
                sort_action="native",
                page_size=15,
                export_format="xlsx"
            )
        ]
        
    except Exception as e:
        return dbc.Alert(f"Error creating summary table: {str(e)}", color="danger")

# Charts callback
@app.callback(
    Output("charts-content", "children"),
    [Input('journey-dropdown', 'value'),
     Input('channel-dropdown', 'value')]
)
def update_charts(selected_journeys, selected_channels):
    try:
        if not selected_journeys or not selected_channels:
            return html.Div("Select filters to view charts", className="text-center text-muted p-4")
        
        # Filter data
        filtered_data = data_loaded[
            (data_loaded['Journey Name'].isin(selected_journeys)) & 
            (data_loaded['Channel'].isin(selected_channels))
        ]
        
        filtered_summary = analyzer.calculate_summary_metrics(filtered_data)
        
        if filtered_summary.empty:
            return html.Div("No data for charts", className="text-center text-warning p-4")
        
        # Chart 1: ROI Performance
        fig1 = px.bar(filtered_summary.nlargest(10, 'ROI'), 
                     x='Journey Name', y='ROI', 
                     title="Top 10 Journeys by ROI")
        fig1.update_layout(height=400, xaxis_tickangle=45)
        
        # Chart 2: Volume vs Conversion
        fig2 = px.scatter(filtered_summary, 
                         x='Sent', y='Unique Click-Through Conversions',
                         size='ROI', hover_name='Journey Name',
                         title="Messages Sent vs Conversions")
        fig2.update_layout(height=400)
        
        return dbc.Row([
            dbc.Col([dcc.Graph(figure=fig1)], width=6),
            dbc.Col([dcc.Graph(figure=fig2)], width=6)
        ])
        
    except Exception as e:
        return dbc.Alert(f"Error creating charts: {str(e)}", color="danger")

# Channel breakdown callback
@app.callback(
    Output("channel-breakdown", "children"),
    [Input('journey-dropdown', 'value'),
     Input('channel-dropdown', 'value')]
)
def update_channel_breakdown(selected_journeys, selected_channels):
    try:
        if not selected_journeys or not selected_channels:
            return html.Div("Select filters to view channel breakdown", className="text-center text-muted p-4")
        
        # Create simple channel breakdown
        filtered_data = data_loaded[
            (data_loaded['Journey Name'].isin(selected_journeys)) & 
            (data_loaded['Channel'].isin(selected_channels))
        ]
        
        channel_summary = filtered_data.groupby(['Journey Name', 'Channel']).agg({
            'Sent': 'sum',
            'Delivered': 'sum',
            'Unique Click-Through Conversions': 'sum'
        }).reset_index()
        
        return [
            html.H5(f"📡 Channel breakdown for selected data", className="mb-3"),
            dash_table.DataTable(
                data=channel_summary.to_dict('records'),
                columns=[{"name": col, "id": col} for col in channel_summary.columns],
                style_table={'overflowX': 'auto'},
                style_cell={'textAlign': 'left', 'padding': '8px'},
                style_header={'backgroundColor': 'rgb(230, 230, 230)', 'fontWeight': 'bold'},
                sort_action="native",
                page_size=20
            )
        ]
        
    except Exception as e:
        return dbc.Alert(f"Error creating channel breakdown: {str(e)}", color="danger")

if __name__ == "__main__":
    local_ip = get_local_ip()
    
    print("🚀 Starting Clean Campaign Analytics Dashboard...")
    print(f"📊 Local URL: http://127.0.0.1:8056")
    print(f"🌐 Network URL: http://{local_ip}:8056")
    print("=" * 50)
    print("Features:")
    print("✅ Clean collapsible interface")
    print("✅ Always visible KPI cards")
    print("✅ Dropdown-style section controls")
    print("✅ Optimized performance")
    print("=" * 50)
    
    try:
        app.run(
            debug=True,
            host='0.0.0.0',
            port=8056,
            dev_tools_hot_reload=True,
            dev_tools_ui=True
        )
    except Exception as e:
        print(f"Error: {e}")
        print("Trying localhost only...")
        app.run(debug=True, host='127.0.0.1', port=8056)

✅ Data loaded successfully: 28750 records
📊 Summary table: 21 journeys
🚀 Starting Clean Campaign Analytics Dashboard...
📊 Local URL: http://127.0.0.1:8056
🌐 Network URL: http://172.16.56.59:8056
Features:
✅ Clean collapsible interface
✅ Always visible KPI cards
✅ Dropdown-style section controls
✅ Optimized performance
Error: HTTPConnectionPool(host='0.0.0.0', port=8056): Max retries exceeded with url: /_alive_2064f1d4-3e91-44a3-a50d-5192a20123eb (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000013165511280>: Failed to establish a new connection: [WinError 10049] The requested address is not valid in its context'))
Trying localhost only...


Error in update_channel_breakdown: 'CampaignAnalyzer' object has no attribute 'calculate_channel_breakdown'
Error in update_charts: 'CampaignAnalyzer' object has no attribute 'calculate_channel_breakdown'


In [ ]:
import pandas as pd
import numpy as np
import warnings
import plotly.graph_objects as go
import plotly.express as px
import dash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
import socket

warnings.filterwarnings('ignore')

# Configuration Variables
AOV = 1500  
CHANNEL_COSTS = {
    'Email': 0.02,
    'Push': 0.01, 
    'SMS': 0.11,
    'WhatsApp': 0.11
}

class CampaignAnalyzer:
    def __init__(self, file_path, aov=AOV, channel_costs=CHANNEL_COSTS):
        self.file_path = file_path
        self.aov = aov
        self.channel_costs = channel_costs
        self.raw_data = None
        
    def load_and_process_data(self):
        """Load and process campaign data"""
        try:
            df = pd.read_csv(self.file_path)
            df = df.fillna(0)
            
            numeric_columns = ['Sent', 'Delivered', 'Unique Impressions', 'Unique Clicks', 
                              'Unique Conversions', 'Unique Click-Through Conversions']
            
            for col in numeric_columns:
                if col in df.columns:
                    df[col] = pd.to_numeric(
                        df[col].astype(str).str.replace(',', '').str.replace('nan', '0'), 
                        errors='coerce'
                    ).fillna(0)
            
            if 'Journey Name' in df.columns:
                df['Journey Name'] = df['Journey Name'].astype(str)
            
            if 'Channel' in df.columns:
                df['Channel'] = df['Channel'].astype(str)
            
            if 'Status' in df.columns:
                df['Status'] = df['Status'].astype(str)
                
            self.raw_data = df
            return df
        except Exception as e:
            print(f"Error loading data: {e}")
            return None
    
    def calculate_summary_metrics(self, filtered_df=None):
        """Calculate main summary table metrics"""
        df = filtered_df if filtered_df is not None else self.raw_data
        
        if df is None or df.empty:
            return pd.DataFrame()
            
        try:
            grouped = df.groupby('Journey Name').agg({
                'Campaign ID': 'count',
                'Sent': 'sum',
                'Delivered': 'sum', 
                'Unique Impressions': 'sum',
                'Unique Clicks': 'sum',
                'Unique Conversions': 'sum',
                'Unique Click-Through Conversions': 'sum'
            }).reset_index()
            
            grouped.rename(columns={'Campaign ID': 'Count of Campaign ID'}, inplace=True)
            
            # Calculate performance metrics
            grouped['Delivery Rate'] = np.where(grouped['Sent'] > 0, 
                                      (grouped['Delivered'] / grouped['Sent'] * 100).round(1), 0)
            
            grouped['CTR'] = np.where(grouped['Delivered'] > 0,
                                       (grouped['Unique Clicks'] / grouped['Delivered'] * 100).round(2), 0)
            
            grouped['Conversion Rate'] = np.where(grouped['Unique Clicks'] > 0,
                                      (grouped['Unique Click-Through Conversions'] / grouped['Unique Clicks'] * 100).round(2), 0)
            
            # Calculate costs and ROI
            grouped['Cost'] = 0
            grouped['GTV'] = grouped['Unique Click-Through Conversions'] * self.aov
            grouped['ROI'] = 0
            
            for idx, row in grouped.iterrows():
                try:
                    journey_data = df[df['Journey Name'] == row['Journey Name']]
                    total_cost = 0
                    for _, campaign in journey_data.iterrows():
                        channel = campaign['Channel']
                        sent = campaign['Sent']
                        if channel in self.channel_costs:
                            total_cost += sent * self.channel_costs[channel]
                    
                    grouped.at[idx, 'Cost'] = total_cost
                    if total_cost > 0:
                        grouped.at[idx, 'ROI'] = (row['GTV'] / total_cost)
                except:
                    pass
            
            return grouped
            
        except Exception as e:
            print(f"Error in calculate_summary_metrics: {e}")
            return pd.DataFrame()

# Initialize the analyzer
try:
    analyzer = CampaignAnalyzer("report-1750749443470_jBRS6FP_Snapmint _in~~c2ab3517.csv")
    data_loaded = analyzer.load_and_process_data()
    
    if data_loaded is not None:
        summary_df = analyzer.calculate_summary_metrics()
        
        print(f" Data loaded successfully: {len(data_loaded)} records")
        print(f" Summary table: {len(summary_df)} journeys")
        
        unique_journeys = sorted([str(j) for j in data_loaded['Journey Name'].unique() if pd.notna(j)])
        unique_channels = sorted([str(c) for c in data_loaded['Channel'].unique() if pd.notna(c)])
        unique_statuses = sorted([str(s) for s in data_loaded['Status'].unique() if pd.notna(s)]) if 'Status' in data_loaded.columns else []
        
        # Date range
        if 'Day' in data_loaded.columns:
            valid_dates = data_loaded['Day'].dropna()
            if not valid_dates.empty:
                min_date = valid_dates.min()
                max_date = valid_dates.max()
            else:
                min_date = max_date = None
        else:
            min_date = max_date = None
            
    else:
        print("❌ Failed to load data")
        summary_df = pd.DataFrame()
        unique_journeys = []
        unique_channels = []
        unique_statuses = []
        min_date = max_date = None
        
except Exception as e:
    print(f"❌ Error initializing analyzer: {e}")
    summary_df = pd.DataFrame()
    unique_journeys = []
    unique_channels = []
    unique_statuses = []
    min_date = max_date = None

# Initialize Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

def get_local_ip():
    """Get the local IP address of the machine"""
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        s.connect(("8.8.8.8", 80))
        local_ip = s.getsockname()[0]
        s.close()
        return local_ip
    except Exception:
        return "127.0.0.1"

# Define the layout - KEEPING YOUR EXISTING FILTER CONTROLS
app.layout = dbc.Container([
    # Header
    dbc.Row([
        dbc.Col([
            html.H1(" Campaign Performance Analytics Dashboard", 
                   className="text-center mb-4 text-primary"),
            html.Hr()
        ])
    ]),
    
    # KEEP YOUR EXISTING FILTER CONTROLS SECTION EXACTLY AS IS
    dbc.Card([
        dbc.CardHeader([
            html.H4("🔍 Filter Controls", className="mb-0")
        ]),
        dbc.CardBody([
            dbc.Row([
                # Journey Selection - UNCHANGED
                dbc.Col([
                    html.Label("📋 Select Journeys:", className="fw-bold"),
                    html.Div([
                        dbc.Button("Select All", id="select-all-journeys", size="sm", color="outline-primary", className="me-2 mb-2"),
                        dbc.Button("Clear All", id="clear-all-journeys", size="sm", color="outline-secondary", className="me-2 mb-2"),
                        dbc.Button("Top 10 ROI", id="top-roi-journeys", size="sm", color="outline-success", className="mb-2"),
                    ]),
                    dcc.Checklist(
                        id='journey-checklist',
                        options=[{'label': journey, 'value': journey} for journey in unique_journeys],
                        value=unique_journeys[:10] if len(unique_journeys) > 10 else unique_journeys,
                        style={'maxHeight': '200px', 'overflowY': 'scroll', 'border': '1px solid #ccc', 'padding': '10px'},
                        className="mt-2"
                    )
                ], width=4),
                
                # Channel Selection - UNCHANGED
                dbc.Col([
                    html.Label("📡 Select Channels:", className="fw-bold"),
                    html.Div([
                        dbc.Button("Select All", id="select-all-channels", size="sm", color="outline-primary", className="me-2 mb-2"),
                        dbc.Button("Clear All", id="clear-all-channels", size="sm", color="outline-secondary", className="mb-2"),
                    ]),
                    dcc.Checklist(
                        id='channel-checklist',
                        options=[{'label': channel, 'value': channel} for channel in unique_channels],
                        value=unique_channels,
                        className="mt-2"
                    )
                ], width=2),
                
                # Status Selection - UNCHANGED
                dbc.Col([
                    html.Label(" Campaign Status:", className="fw-bold"),
                    dcc.Checklist(
                        id='status-checklist',
                        options=[{'label': status, 'value': status} for status in unique_statuses],
                        value=unique_statuses,
                        className="mt-2"
                    )
                ], width=2),
                
                # Date Range and Performance Filters - UNCHANGED
                dbc.Col([
                    html.Label("📅 Date Range:", className="fw-bold"),
                    dcc.DatePickerRange(
                        id='date-picker-range',
                        start_date=min_date,
                        end_date=max_date,
                        display_format='YYYY-MM-DD',
                        className="mb-3"
                    ) if min_date and max_date else html.P("No date data available"),
                    
                    html.Label("🎯 Performance Filter:", className="fw-bold"),
                    dcc.Dropdown(
                        id='performance-filter',
                        options=[
                            {'label': 'All Campaigns', 'value': 'all'},
                            {'label': 'High ROI (>10x)', 'value': 'high_roi'},
                            {'label': 'High CTR (>2%)', 'value': 'high_ctr'},
                            {'label': 'High Conversion (>1%)', 'value': 'high_conv'},
                            {'label': 'Low Performers', 'value': 'low_perf'}
                        ],
                        value='all',
                        className="mt-2"
                    )
                ], width=4),
            ]),
            
            html.Hr(),
            
            # Quick Filter Buttons - UNCHANGED
            dbc.Row([
                dbc.Col([
                    html.Label("⚡ Quick Filters:", className="fw-bold"),
                    html.Div([
                        dbc.Button("🏆 Top Performers", id="quick-top-performers", size="sm", color="success", className="me-2 mb-2"),
                        dbc.Button("📧 Email Only", id="quick-email-only", size="sm", color="info", className="me-2 mb-2"),
                        dbc.Button("📱 Push Only", id="quick-push-only", size="sm", color="warning", className="me-2 mb-2"),
                        dbc.Button("💰 High Value", id="quick-high-value", size="sm", color="primary", className="me-2 mb-2"),
                        dbc.Button("🔄 Reset All", id="reset-filters", size="sm", color="secondary", className="mb-2"),
                    ])
                ])
            ])
        ])
    ], className="mb-4"),
    
    # Always visible KPI Cards
    html.Div(id="kpi-cards", className="mb-4"),
    
    # NEW: COLLAPSIBLE SECTIONS FOR CONTENT (NOT FILTERS)
    html.Div([
        
        # 1. Summary Table Section - COLLAPSIBLE
        dbc.Card([
            dbc.CardHeader([
                dbc.Button(
                    "📋 Summary Table",
                    id="collapse-table-btn",
                    color="primary",
                    className="btn-block text-left w-100",
                    style={"border": "none", "background": "none", "color": "inherit", "font-weight": "bold", "text-align": "left"}
                )
            ]),
            dbc.Collapse([
                dbc.CardBody([
                    html.Div(id="summary-table")
                ])
            ], id="collapse-table", is_open=True)  # Open by default
        ], className="mb-3"),
        
        # 2. Charts Section - COLLAPSIBLE
        dbc.Card([
            dbc.CardHeader([
                dbc.Button(
                    "📈 Performance Charts",
                    id="collapse-charts-btn",
                    color="success",
                    className="btn-block text-left w-100",
                    style={"border": "none", "background": "none", "color": "inherit", "font-weight": "bold", "text-align": "left"}
                )
            ]),
            dbc.Collapse([
                dbc.CardBody([
                    html.Div(id="charts-content")
                ])
            ], id="collapse-charts", is_open=False)
        ], className="mb-3"),
        
        # 3. Channel Breakdown Section - COLLAPSIBLE
        dbc.Card([
            dbc.CardHeader([
                dbc.Button(
                    "📡 Channel Breakdown",
                    id="collapse-channels-btn",
                    color="info",
                    className="btn-block text-left w-100",
                    style={"border": "none", "background": "none", "color": "inherit", "font-weight": "bold", "text-align": "left"}
                )
            ]),
            dbc.Collapse([
                dbc.CardBody([
                    html.Div(id="channel-breakdown")
                ])
            ], id="collapse-channels", is_open=False)
        ], className="mb-3"),
        
        # 4. Campaign Details Section - COLLAPSIBLE
        dbc.Card([
            dbc.CardHeader([
                dbc.Button(
                    " Campaign Details",
                    id="collapse-details-btn",
                    color="warning",
                    className="btn-block text-left w-100",
                    style={"border": "none", "background": "none", "color": "inherit", "font-weight": "bold", "text-align": "left"}
                )
            ]),
            dbc.Collapse([
                dbc.CardBody([
                    html.Div(id="campaign-details")
                ])
            ], id="collapse-details", is_open=False)
        ], className="mb-3"),
        
    ])
    
], fluid=True)

# Collapse button callbacks - ONLY FOR CONTENT SECTIONS
@app.callback(
    Output("collapse-table", "is_open"),
    [Input("collapse-table-btn", "n_clicks")],
    [State("collapse-table", "is_open")],
)
def toggle_table_collapse(n, is_open):
    if n:
        return not is_open
    return is_open

@app.callback(
    Output("collapse-charts", "is_open"),
    [Input("collapse-charts-btn", "n_clicks")],
    [State("collapse-charts", "is_open")],
)
def toggle_charts_collapse(n, is_open):
    if n:
        return not is_open
    return is_open

@app.callback(
    Output("collapse-channels", "is_open"),
    [Input("collapse-channels-btn", "n_clicks")],
    [State("collapse-channels", "is_open")],
)
def toggle_channels_collapse(n, is_open):
    if n:
        return not is_open
    return is_open

@app.callback(
    Output("collapse-details", "is_open"),
    [Input("collapse-details-btn", "n_clicks")],
    [State("collapse-details", "is_open")],
)
def toggle_details_collapse(n, is_open):
    if n:
        return not is_open
    return is_open

# KEEP ALL YOUR EXISTING FILTER CALLBACKS UNCHANGED
@app.callback(
    [Output('journey-checklist', 'value'),
     Output('channel-checklist', 'value')],
    [Input('select-all-journeys', 'n_clicks'),
     Input('clear-all-journeys', 'n_clicks'),
     Input('top-roi-journeys', 'n_clicks'),
     Input('select-all-channels', 'n_clicks'),
     Input('clear-all-channels', 'n_clicks'),
     Input('quick-top-performers', 'n_clicks'),
     Input('quick-email-only', 'n_clicks'),
     Input('quick-push-only', 'n_clicks'),
     Input('quick-high-value', 'n_clicks'),
     Input('reset-filters', 'n_clicks')],
    [State('journey-checklist', 'value'),
     State('channel-checklist', 'value')],
    prevent_initial_call=True
)
def update_filters(select_all_j, clear_all_j, top_roi_j, select_all_c, clear_all_c,
                  quick_top, quick_email, quick_push, quick_value, reset_all,
                  current_journeys, current_channels):
    
    try:
        ctx = dash.callback_context
        if not ctx.triggered:
            return current_journeys or [], current_channels or []
        
        button_id = ctx.triggered[0]['prop_id'].split('.')[0]
        
        if button_id == 'select-all-journeys':
            return unique_journeys, current_channels or []
        elif button_id == 'clear-all-journeys':
            return [], current_channels or []
        elif button_id == 'top-roi-journeys':
            if not summary_df.empty and 'ROI' in summary_df.columns:
                try:
                    top_journeys = summary_df.nlargest(10, 'ROI')['Journey Name'].tolist()
                    return top_journeys, current_channels or []
                except:
                    pass
            return current_journeys or [], current_channels or []
        elif button_id == 'select-all-channels':
            return current_journeys or [], unique_channels
        elif button_id == 'clear-all-channels':
            return current_journeys or [], []
        elif button_id == 'quick-top-performers':
            if not summary_df.empty and 'ROI' in summary_df.columns:
                try:
                    top_journeys = summary_df.nlargest(5, 'ROI')['Journey Name'].tolist()
                    return top_journeys, current_channels or []
                except:
                    pass
            return current_journeys or [], current_channels or []
        elif button_id == 'quick-email-only':
            return current_journeys or [], ['Email']
        elif button_id == 'quick-push-only':
            return current_journeys or [], ['Push']
        elif button_id == 'quick-high-value':
            if not summary_df.empty and 'GTV' in summary_df.columns:
                try:
                    median_gtv = summary_df['GTV'].median()
                    high_value = summary_df[summary_df['GTV'] > median_gtv]['Journey Name'].tolist()
                    return high_value, current_channels or []
                except:
                    pass
            return current_journeys or [], current_channels or []
        elif button_id == 'reset-filters':
            return unique_journeys, unique_channels
        
        return current_journeys or [], current_channels or []
        
    except Exception as e:
        print(f"Error in update_filters: {e}")
        return current_journeys or [], current_channels or []

# KPI Cards callback - UNCHANGED
@app.callback(
    Output("kpi-cards", "children"),
    [Input('journey-checklist', 'value'),
     Input('channel-checklist', 'value'),
     Input('status-checklist', 'value'),
     Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date'),
     Input('performance-filter', 'value')]
)
def update_kpi_cards(selected_journeys, selected_channels, selected_statuses, start_date, end_date, perf_filter):
    try:
        if not selected_journeys or not selected_channels:
            return dbc.Alert("Please select journeys and channels using the Filter Controls section", 
                           color="info", className="text-center")
        
        # Apply all filters
        filtered_data = data_loaded.copy()
        
        if selected_journeys:
            filtered_data = filtered_data[filtered_data['Journey Name'].isin(selected_journeys)]
        if selected_channels:
            filtered_data = filtered_data[filtered_data['Channel'].isin(selected_channels)]
        if selected_statuses:
            filtered_data = filtered_data[filtered_data['Status'].isin(selected_statuses)]
        if start_date and end_date and 'Day' in filtered_data.columns:
            filtered_data = filtered_data[(filtered_data['Day'] >= start_date) & (filtered_data['Day'] <= end_date)]
        
        if filtered_data.empty:
            return dbc.Alert("No data matches your selection", color="warning")
        
        # Calculate totals
        total_sent = filtered_data['Sent'].sum()
        total_delivered = filtered_data['Delivered'].sum()
        total_clicks = filtered_data['Unique Clicks'].sum()
        total_conversions = filtered_data['Unique Click-Through Conversions'].sum()
        
        # Calculate costs
        total_cost = 0
        for _, row in filtered_data.iterrows():
            if row['Channel'] in CHANNEL_COSTS:
                total_cost += row['Sent'] * CHANNEL_COSTS[row['Channel']]
        
        total_gtv = total_conversions * AOV
        overall_roi = total_gtv / total_cost if total_cost > 0 else 0
        
        return dbc.Row([
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H4(f"{total_sent:,}", className="text-primary text-center"),
                        html.P("Messages Sent", className="text-center mb-0"),
                        html.Small(f"{len(set(filtered_data['Journey Name']))} journeys", className="text-muted text-center d-block")
                    ])
                ])
            ], width=2),
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H4(f"{total_delivered:,}", className="text-success text-center"),
                        html.P("Delivered", className="text-center mb-0"),
                        html.Small(f"{(total_delivered/total_sent*100 if total_sent > 0 else 0):.1f}%", 
                                 className="text-muted text-center d-block")
                    ])
                ])
            ], width=2),
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H4(f"{total_clicks:,}", className="text-info text-center"),
                        html.P("Clicks", className="text-center mb-0"),
                        html.Small(f"{(total_clicks/total_delivered*100 if total_delivered > 0 else 0):.2f}%", 
                                 className="text-muted text-center d-block")
                    ])
                ])
            ], width=2),
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H4(f"{total_conversions:,}", className="text-warning text-center"),
                        html.P("Conversions", className="text-center mb-0"),
                        html.Small(f"{(total_conversions/total_clicks*100 if total_clicks > 0 else 0):.2f}%", 
                                 className="text-muted text-center d-block")
                    ])
                ])
            ], width=2),
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H4(f"₹{total_cost:,.0f}", className="text-danger text-center"),
                        html.P("Total Cost", className="text-center mb-0")
                    ])
                ])
            ], width=2),
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H4(f"{overall_roi:.1f}x", className="text-dark text-center"),
                        html.P("ROI", className="text-center mb-0"),
                        html.Small(f"₹{total_gtv:,.0f} GTV", 
                                 className="text-muted text-center d-block")
                    ])
                ])
            ], width=2),
        ])
        
    except Exception as e:
        return dbc.Alert(f"Error calculating metrics: {str(e)}", color="danger")

# Summary table callback
@app.callback(
    Output("summary-table", "children"),
    [Input('journey-checklist', 'value'),
     Input('channel-checklist', 'value'),
     Input('status-checklist', 'value'),
     Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date'),
     Input('performance-filter', 'value')]
)
def update_summary_table(selected_journeys, selected_channels, selected_statuses, start_date, end_date, perf_filter):
    try:
        if not selected_journeys or not selected_channels:
            return html.Div("Select filters to view summary table", className="text-center text-muted p-4")
        
        # Apply all filters
        filtered_data = data_loaded.copy()
        
        if selected_journeys:
            filtered_data = filtered_data[filtered_data['Journey Name'].isin(selected_journeys)]
        if selected_channels:
            filtered_data = filtered_data[filtered_data['Channel'].isin(selected_channels)]
        if selected_statuses:
            filtered_data = filtered_data[filtered_data['Status'].isin(selected_statuses)]
        if start_date and end_date and 'Day' in filtered_data.columns:
            filtered_data = filtered_data[(filtered_data['Day'] >= start_date) & (filtered_data['Day'] <= end_date)]
        
        if filtered_data.empty:
            return html.Div("No data matches your selection", className="text-center text-warning p-4")
        
        # Get summary for selected data
        filtered_summary = analyzer.calculate_summary_metrics(filtered_data)
        
        # Apply performance filter
        if perf_filter == 'high_roi' and 'ROI' in filtered_summary.columns:
            filtered_summary = filtered_summary[filtered_summary['ROI'] > 10]
        elif perf_filter == 'high_ctr' and 'CTR' in filtered_summary.columns:
            filtered_summary = filtered_summary[filtered_summary['CTR'] > 2]
        elif perf_filter == 'high_conv' and 'Conversion Rate' in filtered_summary.columns:
            filtered_summary = filtered_summary[filtered_summary['Conversion Rate'] > 1]
        elif perf_filter == 'low_perf' and 'ROI' in filtered_summary.columns:
            filtered_summary = filtered_summary[filtered_summary['ROI'] < 5]
        
        if filtered_summary.empty:
            return html.Div("No summary data available for current filters", className="text-center text-warning p-4")
        
        # Format display columns
        display_df = filtered_summary[[
            'Journey Name', 'Count of Campaign ID', 'Sent', 'Delivered', 
            'Unique Clicks', 'Unique Click-Through Conversions', 
            'Delivery Rate', 'CTR', 'Conversion Rate', 'Cost', 'GTV', 'ROI'
        ]].copy()
        
        # Format numbers
        for col in ['Sent', 'Delivered', 'Unique Clicks', 'Unique Click-Through Conversions']:
            display_df[col] = display_df[col].apply(lambda x: f"{int(x):,}")
        
        for col in ['Delivery Rate', 'CTR', 'Conversion Rate']:
            display_df[col] = display_df[col].apply(lambda x: f"{x:.1f}%")
        
        display_df['Cost'] = display_df['Cost'].apply(lambda x: f"₹{x:,.0f}")
        display_df['GTV'] = display_df['GTV'].apply(lambda x: f"₹{x:,.0f}")
        display_df['ROI'] = display_df['ROI'].apply(lambda x: f"{x:.1f}x")
        
        return [
            html.H5(f" Summary for {len(display_df)} filtered journeys", className="mb-3"),
            dash_table.DataTable(
                data=display_df.to_dict('records'),
                columns=[{"name": col, "id": col} for col in display_df.columns],
                style_table={'overflowX': 'auto'},
                style_cell={'textAlign': 'left', 'padding': '10px'},
                style_header={'backgroundColor': 'rgb(230, 230, 230)', 'fontWeight': 'bold'},
                sort_action="native",
                page_size=15,
                export_format="xlsx"
            )
        ]
        
    except Exception as e:
        return dbc.Alert(f"Error creating summary table: {str(e)}", color="danger")

# Charts callback
@app.callback(
    Output("charts-content", "children"),
    [Input('journey-checklist', 'value'),
     Input('channel-checklist', 'value'),
     Input('status-checklist', 'value'),
     Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date'),
     Input('performance-filter', 'value')]
)
def update_charts(selected_journeys, selected_channels, selected_statuses, start_date, end_date, perf_filter):
    try:
        if not selected_journeys or not selected_channels:
            return html.Div("Select filters to view charts", className="text-center text-muted p-4")
        
        # Apply filters
        filtered_data = data_loaded.copy()
        
        if selected_journeys:
            filtered_data = filtered_data[filtered_data['Journey Name'].isin(selected_journeys)]
        if selected_channels:
            filtered_data = filtered_data[filtered_data['Channel'].isin(selected_channels)]
        if selected_statuses:
            filtered_data = filtered_data[filtered_data['Status'].isin(selected_statuses)]
        if start_date and end_date and 'Day' in filtered_data.columns:
            filtered_data = filtered_data[(filtered_data['Day'] >= start_date) & (filtered_data['Day'] <= end_date)]
        
        filtered_summary = analyzer.calculate_summary_metrics(filtered_data)
        
        if filtered_summary.empty:
            return html.Div("No data for charts", className="text-center text-warning p-4")
        
        # Chart 1: ROI Performance
        fig1 = px.bar(filtered_summary.nlargest(10, 'ROI'), 
                     x='Journey Name', y='ROI', 
                     title="Top 10 Journeys by ROI")
        fig1.update_layout(height=400, xaxis_tickangle=45)
        
        # Chart 2: Volume vs Conversion
        fig2 = px.scatter(filtered_summary, 
                         x='Sent', y='Unique Click-Through Conversions',
                         size='ROI', hover_name='Journey Name',
                         title="Messages Sent vs Conversions")
        fig2.update_layout(height=400)
        
        return dbc.Row([
            dbc.Col([dcc.Graph(figure=fig1)], width=6),
            dbc.Col([dcc.Graph(figure=fig2)], width=6)
        ])
        
    except Exception as e:
        return dbc.Alert(f"Error creating charts: {str(e)}", color="danger")

# Channel breakdown callback
@app.callback(
    Output("channel-breakdown", "children"),
    [Input('journey-checklist', 'value'),
     Input('channel-checklist', 'value'),
     Input('status-checklist', 'value'),
     Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date')]
)
def update_channel_breakdown(selected_journeys, selected_channels, selected_statuses, start_date, end_date):
    try:
        if not selected_journeys or not selected_channels:
            return html.Div("Select filters to view channel breakdown", className="text-center text-muted p-4")
        
        # Apply filters
        filtered_data = data_loaded.copy()
        
        if selected_journeys:
            filtered_data = filtered_data[filtered_data['Journey Name'].isin(selected_journeys)]
        if selected_channels:
            filtered_data = filtered_data[filtered_data['Channel'].isin(selected_channels)]
        if selected_statuses:
            filtered_data = filtered_data[filtered_data['Status'].isin(selected_statuses)]
        if start_date and end_date and 'Day' in filtered_data.columns:
            filtered_data = filtered_data[(filtered_data['Day'] >= start_date) & (filtered_data['Day'] <= end_date)]
        
        channel_summary = filtered_data.groupby(['Journey Name', 'Channel']).agg({
            'Sent': 'sum',
            'Delivered': 'sum',
            'Unique Click-Through Conversions': 'sum'
        }).reset_index()
        
        # Format numbers
        for col in ['Sent', 'Delivered', 'Unique Click-Through Conversions']:
            channel_summary[col] = channel_summary[col].apply(lambda x: f"{int(x):,}")
        
        return [
            html.H5(f"📡 Channel breakdown for {len(set(filtered_data['Journey Name']))} filtered journeys", className="mb-3"),
            dash_table.DataTable(
                data=channel_summary.to_dict('records'),
                columns=[{"name": col, "id": col} for col in channel_summary.columns],
                style_table={'overflowX': 'auto'},
                style_cell={'textAlign': 'left', 'padding': '8px'},
                style_header={'backgroundColor': 'rgb(230, 230, 230)', 'fontWeight': 'bold'},
                sort_action="native",
                page_size=20,
                export_format="xlsx"
            )
        ]
        
    except Exception as e:
        return dbc.Alert(f"Error creating channel breakdown: {str(e)}", color="danger")

# Campaign details callback
@app.callback(
    Output("campaign-details", "children"),
    [Input('journey-checklist', 'value'),
     Input('channel-checklist', 'value'),
     Input('status-checklist', 'value'),
     Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date')]
)
def update_campaign_details(selected_journeys, selected_channels, selected_statuses, start_date, end_date):
    try:
        if not selected_journeys or not selected_channels:
            return html.Div("Select filters to view campaign details", className="text-center text-muted p-4")
        
        # Apply filters
        filtered_data = data_loaded.copy()
        
        if selected_journeys:
            filtered_data = filtered_data[filtered_data['Journey Name'].isin(selected_journeys)]
        if selected_channels:
            filtered_data = filtered_data[filtered_data['Channel'].isin(selected_channels)]
        if selected_statuses:
            filtered_data = filtered_data[filtered_data['Status'].isin(selected_statuses)]
        if start_date and end_date and 'Day' in filtered_data.columns:
            filtered_data = filtered_data[(filtered_data['Day'] >= start_date) & (filtered_data['Day'] <= end_date)]
        
        # Select relevant columns for campaign details
        detail_cols = ['Campaign Name', 'Journey Name', 'Channel', 'Status', 'Sent', 'Delivered', 'Unique Click-Through Conversions']
        available_detail_cols = [col for col in detail_cols if col in filtered_data.columns]
        
        campaign_details = filtered_data[available_detail_cols].copy()
        
        # Format numeric columns
        for col in ['Sent', 'Delivered', 'Unique Click-Through Conversions']:
            if col in campaign_details.columns:
                campaign_details[col] = campaign_details[col].apply(lambda x: f"{int(x):,}" if pd.notna(x) else "0")
        
        return [
            html.H5(f" Campaign Details - {len(campaign_details)} campaigns", className="mb-3"),
            dash_table.DataTable(
                data=campaign_details.to_dict('records'),
                columns=[{"name": i, "id": i} for i in campaign_details.columns],
                style_table={'overflowX': 'auto'},
                style_cell={'textAlign': 'left', 'padding': '8px', 'fontSize': '12px'},
                style_header={'backgroundColor': 'rgb(230, 230, 230)', 'fontWeight': 'bold'},
                sort_action="native",
                filter_action="native",
                page_size=25,
                export_format="xlsx"
            )
        ]
        
    except Exception as e:
        return dbc.Alert(f"Error creating campaign details: {str(e)}", color="danger")

if __name__ == "__main__":
    local_ip = get_local_ip()
    
    print("Starting Clean Campaign Analytics Dashboard...")
    print(f" Local URL: http://127.0.0.1:8057")
    print(f" Network URL: http://{local_ip}:8057")
    print("=" * 50)
    print("Features:")
    print(" Keep existing filter controls unchanged")
    print(" Collapsible content sections (tables, charts, etc.)")
    print(" Always visible KPI cards")
    print(" All existing functionality preserved")
    print("=" * 50)
    
    try:
        app.run(
            debug=True,
            host='0.0.0.0',
            port=8058,
            dev_tools_hot_reload=True,
            dev_tools_ui=True
        )
    except Exception as e:
        print(f"Error: {e}")
        print("Trying localhost only...")
        app.run(debug=True, host='127.0.0.1', port=8058)

 Data loaded successfully: 28750 records
 Summary table: 21 journeys
Starting Clean Campaign Analytics Dashboard...
 Local URL: http://127.0.0.1:8057
 Network URL: http://172.16.56.59:8057
Features:
 Keep existing filter controls unchanged
 Collapsible content sections (tables, charts, etc.)
 Always visible KPI cards
 All existing functionality preserved
Error: HTTPConnectionPool(host='0.0.0.0', port=8058): Max retries exceeded with url: /_alive_13003cbf-647f-4e4f-a5af-c226ea982ad1 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002DD9B4A2F90>: Failed to establish a new connection: [WinError 10049] The requested address is not valid in its context'))
Trying localhost only...


: 